In [ ]:
from fastapi import FastAPI, HTTPException, BackgroundTasks
from pydantic import BaseModel
import asyncio
import aiohttp
import time
from typing import Dict, List, Optional, Set
from collections import defaultdict, deque
import statistics
from datetime import datetime, timedelta
import logging

app = FastAPI(title="Onchain Anomaly Tracker")

# Configuration
ETHERSCAN_API_KEY = "YOUR_ETHERSCAN_API_KEY"
WEBACY_API_KEY = "YOUR_WEBACY_API_KEY"
RATE_LIMIT_DELAY = 0.2  # seconds between API calls

# Global caches and state
contract_cache = {}
address_monitor = {}  # Track addresses being monitored
anomaly_cache = deque(maxlen=10000)  # Store recent anomalies

class AddressAnalysis(BaseModel):
    address: str
    address_type: str
    anomaly_score: int
    risk_level: str
    flags: List[str]
    webacy_sanctioned: bool
    webacy_threat_score: Optional[float] = None

class AnomalyDetector:
    def __init__(self):
        self.session = None
        self.monitored_addresses: Set[str] = set()
        
    async def init_session(self):
        if not self.session:
            self.session = aiohttp.ClientSession()
    
    async def close_session(self):
        if self.session:
            await self.session.close()

    async def is_contract(self, address: str) -> bool:
        """Cached contract detection"""
        if address in contract_cache:
            return contract_cache[address]
        
        await self.init_session()
        url = f"https://api.etherscan.io/api?module=proxy&action=eth_getCode&address={address}&tag=latest&apikey={ETHERSCAN_API_KEY}"
        
        try:
            async with self.session.get(url) as response:
                data = await response.json()
                result = data.get("result", "0x") != "0x"
                contract_cache[address] = result
                await asyncio.sleep(RATE_LIMIT_DELAY)
                return result
        except Exception as e:
            logging.error(f"Error checking contract status for {address}: {e}")
            return False

    async def fetch_transactions_batch(self, address: str, action: str = "txlist", max_pages: int = 2) -> List[Dict]:
        """Optimized transaction fetching with recent data focus"""
        await self.init_session()
        transactions = []
        
        # Get only recent transactions (last 1000 blocks ~4 hours)
        latest_block = await self.get_latest_block()
        start_block = max(0, latest_block - 1000)
        
        for page in range(1, max_pages + 1):
            url = f"https://api.etherscan.io/v2/api?chainid=1&module=account&action={action}&address={address}&startblock={start_block}&endblock=latest&page={page}&offset=1000&sort=desc&apikey={ETHERSCAN_API_KEY}"
            
            try:
                async with self.session.get(url) as response:
                    data = await response.json()
                    txs = data.get("result", [])
                    
                    if not txs or txs == "No transactions found":
                        break
                        
                    transactions.extend(txs)
                    await asyncio.sleep(RATE_LIMIT_DELAY)
                    
                    # If we get less than 1000, we've got all recent data
                    if len(txs) < 1000:
                        break
                        
            except Exception as e:
                logging.error(f"Error fetching {action} for {address}: {e}")
                break
        
        return transactions

    async def get_latest_block(self) -> int:
        """Get current block number"""
        await self.init_session()
        url = f"https://api.etherscan.io/api?module=proxy&action=eth_blockNumber&apikey={ETHERSCAN_API_KEY}"
        
        try:
            async with self.session.get(url) as response:
                data = await response.json()
                return int(data.get("result", "0x0"), 16)
        except:
            return 0

    def analyze_transaction_frequency(self, txs: List[Dict], window_minutes: int = 10) -> Dict:
        """Detect unusual transaction frequency patterns"""
        if not txs:
            return {"frequency_anomaly": False}
            
        timestamps = [int(tx['timeStamp']) for tx in txs]
        timestamps.sort()
        
        # Group by time windows
        window_seconds = window_minutes * 60
        freq_buckets = defaultdict(int)
        
        for ts in timestamps:
            bucket = ts // window_seconds
            freq_buckets[bucket] += 1
        
        if len(freq_buckets) < 2:
            return {"frequency_anomaly": False}
        
        frequencies = list(freq_buckets.values())
        avg_freq = statistics.mean(frequencies)
        max_freq = max(frequencies)
        
        # Anomaly if max frequency is 5x above average
        is_anomaly = max_freq > (avg_freq * 5) and max_freq > 20
        
        return {
            "frequency_anomaly": is_anomaly,
            "max_frequency": max_freq,
            "avg_frequency": avg_freq,
            "frequency_ratio": max_freq / avg_freq if avg_freq > 0 else 0
        }

    def analyze_gas_patterns(self, txs: List[Dict]) -> Dict:
        """Detect gas price manipulation and MEV activity"""
        if not txs:
            return {"gas_anomaly": False}
        
        gas_prices = [int(tx['gasPrice']) for tx in txs if tx.get('gasPrice', '0') != '0']
        gas_used = [int(tx['gasUsed']) for tx in txs if tx.get('gasUsed', '0') != '0']
        failed_txs = [tx for tx in txs if tx.get('isError') == '1']
        
        if not gas_prices:
            return {"gas_anomaly": False}
        
        avg_gas_price = statistics.mean(gas_prices)
        max_gas_price = max(gas_prices)
        min_gas_price = min(gas_prices)
        
        # Detect MEV/sandwich attacks (extremely high gas prices)
        high_gas_count = len([g for g in gas_prices if g > avg_gas_price * 3])
        failed_ratio = len(failed_txs) / len(txs) if txs else 0
        
        gas_variance = (max_gas_price / min_gas_price) if min_gas_price > 0 else 0
        
        is_anomaly = (
            high_gas_count > 5 or  # Multiple high-gas transactions
            failed_ratio > 0.3 or  # High failure rate
            gas_variance > 100      # Extreme gas price variance
        )
        
        return {
            "gas_anomaly": is_anomaly,
            "high_gas_count": high_gas_count,
            "failed_ratio": failed_ratio,
            "gas_variance": gas_variance,
            "avg_gas_price": avg_gas_price
        }

    def analyze_value_patterns(self, txs: List[Dict]) -> Dict:
        """Detect suspicious value transfer patterns"""
        if not txs:
            return {"value_anomaly": False}
        
        values = [int(tx['value']) for tx in txs if int(tx.get('value', '0')) > 0]
        
        if not values:
            return {"value_anomaly": False}
        
        # Detect round number bias (potential money laundering)
        round_values = [v for v in values if v % (10**18) == 0]  # Exact ETH amounts
        round_ratio = len(round_values) / len(values)
        
        # Detect dust attacks
        dust_threshold = 10**15  # 0.001 ETH
        dust_count = len([v for v in values if v < dust_threshold])
        
        # Detect value concentration
        total_value = sum(values)
        max_value = max(values)
        concentration = max_value / total_value if total_value > 0 else 0
        
        is_anomaly = (
            round_ratio > 0.7 or      # Too many round numbers
            dust_count > 50 or        # Dust attack
            concentration > 0.9       # Single large transaction dominates
        )
        
        return {
            "value_anomaly": is_anomaly,
            "round_number_ratio": round_ratio,
            "dust_attack_count": dust_count,
            "value_concentration": concentration
        }

    def detect_bot_patterns(self, txs: List[Dict]) -> Dict:
        """Detect automated/bot activity patterns"""
        if len(txs) < 5:
            return {"bot_detected": False}
        
        timestamps = [int(tx['timeStamp']) for tx in txs]
        timestamps.sort()
        
        # Calculate intervals between transactions
        intervals = [timestamps[i+1] - timestamps[i] for i in range(len(timestamps)-1)]
        
        if not intervals:
            return {"bot_detected": False}
        
        # Look for regular patterns (±3 seconds)
        regular_intervals = []
        for interval in intervals:
            for target in [10, 15, 30, 60]:  # Common bot intervals
                if abs(interval - target) <= 3:
                    regular_intervals.append(interval)
                    break
        
        regularity_ratio = len(regular_intervals) / len(intervals)
        
        # Check for identical gas prices (bot signature)
        gas_prices = [tx.get('gasPrice', '0') for tx in txs]
        unique_gas_prices = len(set(gas_prices))
        gas_uniformity = 1 - (unique_gas_prices / len(gas_prices)) if gas_prices else 0
        
        is_bot = (
            regularity_ratio > 0.8 or     # Very regular timing
            gas_uniformity > 0.9          # Very uniform gas prices
        )
        
        return {
            "bot_detected": is_bot,
            "regularity_ratio": regularity_ratio,
            "gas_uniformity": gas_uniformity,
            "regular_interval_count": len(regular_intervals)
        }

    async def webacy_sanction_check(self, address: str) -> Dict:
        """Quick sanction screening via Webacy"""
        await self.init_session()
        url = f"https://api.webacy.com/addresses/sanctioned/{address}"
        headers = {"accept": "application/json", "x-api-key": WEBACY_API_KEY}
        
        try:
            async with self.session.get(url, headers=headers) as response:
                data = await response.json()
                return {
                    "is_sanctioned": data.get("is_sanctioned", False),
                    "webacy_check": True
                }
        except Exception as e:
            logging.error(f"Webacy sanction check failed for {address}: {e}")
            return {"is_sanctioned": False, "webacy_check": False}

    async def webacy_threat_analysis(self, address: str) -> Dict:
        """Full threat analysis via Webacy if needed"""
        await self.init_session()
        url = f"https://api.webacy.com/addresses/{address}"
        headers = {"accept": "application/json", "x-api-key": WEBACY_API_KEY}
        
        try:
            async with self.session.get(url, headers=headers) as response:
                data = await response.json()
                return {
                    "threat_score": data.get("overallRisk", 0),
                    "risk_level": data.get("riskScore", "Unknown"),
                    "issue_count": data.get("count", 0)
                }
        except Exception as e:
            logging.error(f"Webacy threat analysis failed for {address}: {e}")
            return {"threat_score": 0, "risk_level": "Unknown", "issue_count": 0}

    def calculate_combined_anomaly_score(self, patterns: Dict) -> tuple:
        """Calculate overall anomaly score and flags"""
        score = 0
        flags = []
        
        # Frequency anomalies
        if patterns.get("frequency_anomaly", False):
            score += 35
            flags.append(f"High Frequency Activity ({patterns.get('max_frequency', 0)} txs)")
        
        # Gas pattern anomalies
        if patterns.get("gas_anomaly", False):
            score += 25
            flags.append("Suspicious Gas Patterns")
            if patterns.get("failed_ratio", 0) > 0.3:
                flags.append("High Failed Transaction Rate")
        
        # Value pattern anomalies
        if patterns.get("value_anomaly", False):
            score += 20
            if patterns.get("round_number_ratio", 0) > 0.7:
                flags.append("Round Number Bias")
            if patterns.get("dust_attack_count", 0) > 50:
                flags.append("Dust Attack Pattern")
        
        # Bot detection
        if patterns.get("bot_detected", False):
            score += 30
            flags.append("Automated Activity Detected")
        
        # Webacy threat intelligence
        if patterns.get("is_sanctioned", False):
            score += 50
            flags.append("SANCTIONED ADDRESS")
        
        threat_score = patterns.get("threat_score", 0)
        if threat_score > 50:
            score += 40
            flags.append(f"High Threat Score ({threat_score})")
        
        # Determine risk level
        if score >= 80:
            risk_level = "CRITICAL"
        elif score >= 60:
            risk_level = "HIGH"
        elif score >= 40:
            risk_level = "MEDIUM"
        else:
            risk_level = "LOW"
        
        return min(score, 100), risk_level, flags

    async def analyze_address(self, address: str) -> AddressAnalysis:
        """Comprehensive address analysis combining all detection methods"""
        try:
            # Quick sanction check first
            sanction_result = await self.webacy_sanction_check(address)
            
            # Determine address type
            is_contract_addr = await self.is_contract(address)
            addr_type = "Contract" if is_contract_addr else "EOA"
            
            # Fetch recent transaction data
            normal_txs = await self.fetch_transactions_batch(address, "txlist", 2)
            internal_txs = await self.fetch_transactions_batch(address, "txlistinternal", 1)
            token_txs = await self.fetch_transactions_batch(address, "tokentx", 1)
            
            # Analyze patterns
            freq_analysis = self.analyze_transaction_frequency(normal_txs)
            gas_analysis = self.analyze_gas_patterns(normal_txs)
            value_analysis = self.analyze_value_patterns(normal_txs)
            bot_analysis = self.detect_bot_patterns(normal_txs)
            
            # Combine all patterns
            all_patterns = {
                **freq_analysis,
                **gas_analysis,
                **value_analysis,
                **bot_analysis,
                **sanction_result
            }
            
            # Get detailed threat analysis if anomalies detected
            if any([freq_analysis.get("frequency_anomaly"), gas_analysis.get("gas_anomaly"), 
                   value_analysis.get("value_anomaly"), bot_analysis.get("bot_detected"),
                   sanction_result.get("is_sanctioned")]):
                threat_data = await self.webacy_threat_analysis(address)
                all_patterns.update(threat_data)
            
            # Calculate final score
            score, risk_level, flags = self.calculate_combined_anomaly_score(all_patterns)
            
            return AddressAnalysis(
                address=address,
                address_type=addr_type,
                anomaly_score=score,
                risk_level=risk_level,
                flags=flags,
                webacy_sanctioned=sanction_result.get("is_sanctioned", False),
                webacy_threat_score=all_patterns.get("threat_score")
            )
            
        except Exception as e:
            logging.error(f"Analysis failed for {address}: {e}")
            raise HTTPException(status_code=500, detail=f"Analysis failed: {str(e)}")

# Initialize detector
detector = AnomalyDetector()

@app.on_event("startup")
async def startup_event():
    await detector.init_session()

@app.on_event("shutdown")
async def shutdown_event():
    await detector.close_session()

@app.post("/analyze", response_model=AddressAnalysis)
async def analyze_address(address: str):
    """Analyze a single address for anomalies"""
    return await detector.analyze_address(address.lower())

@app.post("/monitor")
async def add_to_monitoring(address: str, background_tasks: BackgroundTasks):
    """Add address to continuous monitoring"""
    address = address.lower()
    detector.monitored_addresses.add(address)
    background_tasks.add_task(continuous_monitoring, address)
    return {"message": f"Address {address} added to monitoring"}

@app.get("/monitored")
async def get_monitored_addresses():
    """Get list of currently monitored addresses"""
    return {"monitored_addresses": list(detector.monitored_addresses)}

async def continuous_monitoring(address: str):
    """Background task for continuous address monitoring"""
    while address in detector.monitored_addresses:
        try:
            analysis = await detector.analyze_address(address)
            
            # Store anomaly if score is significant
            if analysis.anomaly_score > 40:
                anomaly_data = {
                    "timestamp": datetime.now().isoformat(),
                    "address": address,
                    "analysis": analysis.dict()
                }
                anomaly_cache.append(anomaly_data)
                logging.warning(f"ANOMALY DETECTED: {address} - Score: {analysis.anomaly_score}")
            
            # Monitor every 5 minutes
            await asyncio.sleep(300)
            
        except Exception as e:
            logging.error(f"Monitoring error for {address}: {e}")
            await asyncio.sleep(60)  # Retry after 1 minute on error

@app.get("/anomalies")
async def get_recent_anomalies():
    """Get recent detected anomalies"""
    return {"recent_anomalies": list(anomaly_cache)}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
from fastapi import FastAPI, BackgroundTasks, HTTPException
from fastapi.responses import StreamingResponse
import asyncio
import aiohttp
import time
import json
from typing import Dict, List, Optional, Set
from datetime import datetime, timedelta
from pydantic import BaseModel
import logging
from collections import defaultdict

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI(title="Blockchain Anomaly Detection System", version="1.0.0")

# Pydantic models for API responses
class AnomalyResult(BaseModel):
    address: str
    anomaly_score: int
    risk_level: str
    flags: List[str]
    analysis_timestamp: str
    
class TransactionSummary(BaseModel):
    total_transactions: int
    unique_addresses: int
    anomaly_count: int
    high_risk_addresses: List[str]

class BlockchainAnomalyDetector:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.contract_cache: Dict[str, bool] = {}
        self.anomaly_results: Dict[str, Dict] = {}
        self.base_url = "https://api.etherscan.io"
        self.rate_limit_delay = 0.2  # 200ms between requests
        
    async def get_transactions_async(self, address: str, action: str = "txlist", 
                                   chainid: int = 1, offset: int = 1000, 
                                   max_pages: int = 5) -> List[Dict]:
        """Async version of transaction fetching"""
        transactions = []
        
        async with aiohttp.ClientSession() as session:
            for page in range(1, max_pages + 1):
                url = (
                    f"{self.base_url}/v2/api"
                    f"?chainid={chainid}"
                    f"&module=account"
                    f"&action={action}"
                    f"&address={address}"
                    f"&startblock=0"
                    f"&endblock=latest"
                    f"&page={page}"
                    f"&offset={offset}"
                    f"&sort=desc"
                    f"&apikey={self.api_key}"
                )
                
                try:
                    async with session.get(url) as response:
                        data = await response.json()
                        txs = data.get("result", [])
                        
                        if not txs or txs == "No transactions found":
                            break
                            
                        transactions.extend(txs)
                        logger.info(f"Fetched page {page}, got {len(txs)} {action} transactions")
                        
                        # Rate limiting
                        await asyncio.sleep(self.rate_limit_delay)
                        
                except Exception as e:
                    logger.error(f"Error fetching {action} transactions page {page}: {e}")
                    break
                    
        return transactions

    async def is_contract_async(self, address: str) -> bool:
        """Async version of contract checking with caching"""
        if address in self.contract_cache:
            return self.contract_cache[address]
        
        url = f"{self.base_url}/api?module=proxy&action=eth_getCode&address={address}&tag=latest&apikey={self.api_key}"
        
        try:
            async with aiohttp.ClientSession() as session:
                async with session.get(url) as response:
                    data = await response.json()
                    result = data.get("result", "0x") != "0x"
                    self.contract_cache[address] = result
                    await asyncio.sleep(self.rate_limit_delay)
                    return result
        except Exception as e:
            logger.error(f"Error checking contract status for {address}: {e}")
            return False

    async def enrich_transactions(self, transactions: List[Dict]) -> List[Dict]:
        """Add contract/EOA labels and other enrichments"""
        enriched = []
        
        for tx in transactions:
            from_addr = tx.get("from", "")
            to_addr = tx.get("to", "")
            
            # Add address types
            if from_addr:
                tx["from_type"] = "Contract" if await self.is_contract_async(from_addr) else "EOA"
            if to_addr:
                tx["to_type"] = "Contract" if await self.is_contract_async(to_addr) else "EOA"
            
            # Add scaled values for token transactions
            if "tokenDecimal" in tx:
                decimals = int(tx.get("tokenDecimal", 0))
                tx["value_scaled"] = int(tx.get("value", 0)) / (10 ** decimals) if decimals else int(tx.get("value", 0))
            
            enriched.append(tx)
            
        return enriched

    # Anomaly Detection Methods
    def analyze_transaction_frequency(self, txs: List[Dict], window_minutes: int = 60) -> Dict:
        """Detect transaction frequency anomalies"""
        freq_patterns = {}
        for tx in txs:
            timestamp = int(tx.get('timeStamp', 0))
            window_key = timestamp // (window_minutes * 60)
            freq_patterns[window_key] = freq_patterns.get(window_key, 0) + 1
        
        if not freq_patterns:
            return {"anomalies": [], "avg_frequency": 0}
            
        avg_freq = sum(freq_patterns.values()) / len(freq_patterns)
        anomalies = [k for k, v in freq_patterns.items() if v > avg_freq * 5]
        
        return {
            "anomalies": anomalies,
            "avg_frequency": avg_freq,
            "spike_count": len(anomalies)
        }

    def detect_bot_activity(self, txs: List[Dict]) -> Dict:
        """Detect bot-like regular interval patterns"""
        timestamps = [int(tx.get('timeStamp', 0)) for tx in txs]
        timestamps.sort()
        
        if len(timestamps) < 3:
            return {"bot_suspected": False, "regular_interval_count": 0}
            
        intervals = [timestamps[i+1] - timestamps[i] for i in range(len(timestamps)-1)]
        
        # Bot patterns: very regular intervals (±2 seconds)
        regular_intervals = [i for i in intervals if 8 <= i <= 12]  # 10-second bot pattern
        
        return {
            "bot_suspected": len(regular_intervals) > 10,
            "regular_interval_count": len(regular_intervals),
            "total_intervals": len(intervals),
            "regularity_ratio": len(regular_intervals) / len(intervals) if intervals else 0
        }

    def analyze_value_anomalies(self, txs: List[Dict]) -> Dict:
        """Detect unusual value patterns"""
        values = []
        for tx in txs:
            if 'value_scaled' in tx:
                values.append(float(tx['value_scaled']))
            elif 'value' in tx:
                val = int(tx['value'])
                if val > 0:
                    values.append(val)
        
        if not values:
            return {"round_number_ratio": 0, "dust_attack_count": 0, "value_concentration": 0}
        
        avg_value = sum(values) / len(values)
        
        # Detect round number bias (money laundering indicator)
        round_values = [v for v in values if v % 1 == 0 and v > 0]  # Round numbers
        
        # Detect dust attacks
        dust_threshold = 0.001 if 'value_scaled' in txs[0] else 10**15  # 0.001 ETH
        dust_txs = [v for v in values if v < dust_threshold]
        
        return {
            "round_number_ratio": len(round_values) / len(values) if values else 0,
            "dust_attack_count": len(dust_txs),
            "value_concentration": max(values) / avg_value if avg_value > 0 else 0,
            "value_variance": (max(values) - min(values)) / avg_value if avg_value > 0 else 0
        }

    def analyze_gas_patterns(self, txs: List[Dict]) -> Dict:
        """Analyze gas usage patterns for anomalies"""
        gas_prices = [int(tx.get('gasPrice', 0)) for tx in txs if tx.get('gasPrice')]
        gas_used = [int(tx.get('gasUsed', 0)) for tx in txs if tx.get('gasUsed')]
        
        if not gas_prices:
            return {"high_gas_count": 0, "failed_tx_ratio": 0, "gas_price_variance": 0}
            
        avg_gas_price = sum(gas_prices) / len(gas_prices)
        
        # Detect MEV/sandwich attacks: unusually high gas prices
        high_gas_txs = [g for g in gas_prices if g > avg_gas_price * 3]
        
        # Detect failed transaction spams
        failed_txs = [tx for tx in txs if tx.get('isError') == '1']
        
        return {
            "high_gas_count": len(high_gas_txs),
            "failed_tx_ratio": len(failed_txs) / len(txs) if txs else 0,
            "gas_price_variance": max(gas_prices) / min(gas_prices) if gas_prices and min(gas_prices) > 0 else 0,
            "avg_gas_price": avg_gas_price
        }

    def analyze_address_relationships(self, normal_txs: List[Dict], internal_txs: List[Dict], token_txs: List[Dict]) -> Dict:
        """Analyze interaction patterns between addresses"""
        all_addresses = set()
        interactions = {}
        
        # Build interaction graph
        for tx_list, tx_type in [(normal_txs, 'normal'), (internal_txs, 'internal'), (token_txs, 'token')]:
            for tx in tx_list:
                from_addr = tx.get('from', '')
                to_addr = tx.get('to', '')
                
                if from_addr and to_addr:
                    key = (from_addr, to_addr)
                    if key not in interactions:
                        interactions[key] = {'count': 0, 'types': set(), 'values': []}
                    interactions[key]['count'] += 1
                    interactions[key]['types'].add(tx_type)
                    
                    # Add value if available
                    value = tx.get('value_scaled', tx.get('value', 0))
                    if value:
                        interactions[key]['values'].append(float(value))
                    
                    all_addresses.update([from_addr, to_addr])
        
        # Detect suspicious patterns
        circular_flows = self.detect_circular_transactions(interactions)
        high_interaction_pairs = [(k, v['count']) for k, v in interactions.items() if v['count'] > 50]
        
        return {
            "unique_addresses": len(all_addresses),
            "interaction_pairs": len(interactions),
            "circular_flows": len(circular_flows),
            "high_interaction_pairs": len(high_interaction_pairs),
            "avg_interactions_per_pair": sum(v['count'] for v in interactions.values()) / len(interactions) if interactions else 0
        }

    def detect_circular_transactions(self, interactions: Dict) -> List[Dict]:
        """Detect circular transaction patterns"""
        circular = []
        interaction_graph = {}
        
        # Build adjacency list
        for (from_addr, to_addr), data in interactions.items():
            if from_addr not in interaction_graph:
                interaction_graph[from_addr] = []
            interaction_graph[from_addr].append((to_addr, data))
        
        # Look for simple circular patterns (A->B->A)
        for addr_a in interaction_graph:
            for addr_b, data_ab in interaction_graph[addr_a]:
                if addr_b in interaction_graph:
                    for addr_c, data_bc in interaction_graph[addr_b]:
                        if addr_c == addr_a:  # Found circle
                            circular.append({
                                "addresses": [addr_a, addr_b],
                                "ab_count": data_ab['count'],
                                "ba_count": data_bc['count']
                            })
        
        return circular

    def detect_coordinated_patterns(self, normal_txs: List[Dict], internal_txs: List[Dict], token_txs: List[Dict]) -> Dict:
        """Detect coordinated activity across transaction types"""
        timestamps = {
            'normal': set(tx.get('timeStamp', '') for tx in normal_txs),
            'internal': set(tx.get('timeStamp', '') for tx in internal_txs),
            'token': set(tx.get('timeStamp', '') for tx in token_txs)
        }
        
        # Remove empty timestamps
        for key in timestamps:
            timestamps[key].discard('')
        
        all_times = timestamps['normal'] | timestamps['internal'] | timestamps['token']
        coordinated_times = []
        
        for time in all_times:
            types_at_time = []
            if time in timestamps['normal']: types_at_time.append('normal')
            if time in timestamps['internal']: types_at_time.append('internal')  
            if time in timestamps['token']: types_at_time.append('token')
            
            if len(types_at_time) >= 2:
                coordinated_times.append({
                    "timestamp": time,
                    "types": types_at_time
                })
        
        return {
            "coordinated_timestamp_count": len(coordinated_times),
            "coordination_ratio": len(coordinated_times) / len(all_times) if all_times else 0,
            "coordinated_events": coordinated_times[:10]  # Sample of events
        }

    def calculate_anomaly_score(self, address: str, analysis_data: Dict) -> Dict:
        """Calculate overall anomaly score"""
        score = 0
        flags = []
        
        # Bot activity detection
        if analysis_data.get('bot_activity', {}).get('bot_suspected', False):
            score += 30
            flags.append("Bot Activity Detected")
        
        # Value pattern anomalies
        round_ratio = analysis_data.get('value_patterns', {}).get('round_number_ratio', 0)
        if round_ratio > 0.5:
            score += 20
            flags.append("Round Number Bias")
        
        # High dust attack count
        dust_count = analysis_data.get('value_patterns', {}).get('dust_attack_count', 0)
        if dust_count > 100:
            score += 15
            flags.append("Potential Dust Attack")
        
        # Failed transaction spam
        failed_ratio = analysis_data.get('gas_patterns', {}).get('failed_tx_ratio', 0)
        if failed_ratio > 0.2:
            score += 25
            flags.append("High Failed Transaction Rate")
        
        # Coordinated activity
        coord_ratio = analysis_data.get('coordination', {}).get('coordination_ratio', 0)
        if coord_ratio > 0.3:
            score += 40
            flags.append("Coordinated Multi-Type Transactions")
        
        # Frequency spikes
        spike_count = analysis_data.get('frequency', {}).get('spike_count', 0)
        if spike_count > 5:
            score += 20
            flags.append("Transaction Frequency Spikes")
        
        # Circular transaction patterns
        circular_count = analysis_data.get('relationships', {}).get('circular_flows', 0)
        if circular_count > 0:
            score += 35
            flags.append("Circular Transaction Patterns")
        
        # High gas price variance (MEV activity)
        gas_variance = analysis_data.get('gas_patterns', {}).get('gas_price_variance', 0)
        if gas_variance > 10:
            score += 25
            flags.append("Unusual Gas Price Patterns")
        
        return {
            "address": address,
            "anomaly_score": min(score, 100),
            "risk_level": "HIGH" if score > 70 else "MEDIUM" if score > 40 else "LOW",
            "flags": flags,
            "analysis_timestamp": datetime.now().isoformat()
        }

    async def analyze_address_comprehensive(self, address: str) -> Dict:
        """Comprehensive analysis of an address"""
        logger.info(f"Starting comprehensive analysis for address: {address}")
        
        # Fetch all transaction types
        normal_txs = await self.get_transactions_async(address, "txlist")
        internal_txs = await self.get_transactions_async(address, "txlistinternal")  
        token_txs = await self.get_transactions_async(address, "tokentx")
        
        # Enrich transactions
        normal_txs = await self.enrich_transactions(normal_txs)
        internal_txs = await self.enrich_transactions(internal_txs)
        token_txs = await self.enrich_transactions(token_txs)
        
        # Perform all analyses
        analysis_data = {
            "frequency": self.analyze_transaction_frequency(normal_txs + internal_txs + token_txs),
            "bot_activity": self.detect_bot_activity(normal_txs + internal_txs + token_txs),
            "value_patterns": self.analyze_value_anomalies(token_txs + [tx for tx in normal_txs if int(tx.get('value', 0)) > 0]),
            "gas_patterns": self.analyze_gas_patterns(normal_txs),
            "relationships": self.analyze_address_relationships(normal_txs, internal_txs, token_txs),
            "coordination": self.detect_coordinated_patterns(normal_txs, internal_txs, token_txs)
        }
        
        # Calculate anomaly score
        anomaly_result = self.calculate_anomaly_score(address, analysis_data)
        
        # Store results
        self.anomaly_results[address] = {
            "anomaly_result": anomaly_result,
            "detailed_analysis": analysis_data,
            "transaction_counts": {
                "normal": len(normal_txs),
                "internal": len(internal_txs),
                "token": len(token_txs)
            }
        }
        
        logger.info(f"Analysis complete for {address}. Risk level: {anomaly_result['risk_level']}")
        return self.anomaly_results[address]

# Global detector instance
detector = None

@app.on_event("startup")
async def startup_event():
    global detector
    # Replace with your actual API key
    api_key = "YOUR_ETHERSCAN_API_KEY"
    detector = BlockchainAnomalyDetector(api_key)
    logger.info("Blockchain Anomaly Detection System started")

@app.get("/", summary="Health Check")
async def root():
    return {"message": "Blockchain Anomaly Detection System is running"}

@app.post("/analyze/{address}", response_model=AnomalyResult, summary="Analyze Address for Anomalies")
async def analyze_address(address: str):
    """Perform comprehensive anomaly analysis on a blockchain address"""
    if not detector:
        raise HTTPException(status_code=500, detail="Detector not initialized")
    
    try:
        result = await detector.analyze_address_comprehensive(address)
        return AnomalyResult(**result["anomaly_result"])
    except Exception as e:
        logger.error(f"Error analyzing address {address}: {e}")
        raise HTTPException(status_code=500, detail=f"Analysis failed: {str(e)}")

@app.get("/results/{address}", summary="Get Analysis Results")
async def get_analysis_results(address: str):
    """Get detailed analysis results for an address"""
    if not detector or address not in detector.anomaly_results:
        raise HTTPException(status_code=404, detail="Analysis results not found")
    
    return detector.anomaly_results[address]

@app.get("/summary", response_model=TransactionSummary, summary="Get Analysis Summary")
async def get_summary():
    """Get summary of all analyses performed"""
    if not detector:
        raise HTTPException(status_code=500, detail="Detector not initialized")
    
    results = detector.anomaly_results
    high_risk = [addr for addr, data in results.items() 
                 if data["anomaly_result"]["risk_level"] == "HIGH"]
    
    return TransactionSummary(
        total_transactions=sum(
            sum(data["transaction_counts"].values()) 
            for data in results.values()
        ),
        unique_addresses=len(results),
        anomaly_count=len([r for r in results.values() if r["anomaly_result"]["anomaly_score"] > 40]),
        high_risk_addresses=high_risk
    )

@app.get("/batch-analyze", summary="Batch Analyze Multiple Addresses")
async def batch_analyze(addresses: str):
    """Analyze multiple addresses separated by commas"""
    if not detector:
        raise HTTPException(status_code=500, detail="Detector not initialized")
    
    addr_list = [addr.strip() for addr in addresses.split(",") if addr.strip()]
    
    if len(addr_list) > 10:
        raise HTTPException(status_code=400, detail="Maximum 10 addresses allowed per batch")
    
    results = []
    for address in addr_list:
        try:
            result = await detector.analyze_address_comprehensive(address)
            results.append(result["anomaly_result"])
        except Exception as e:
            results.append({
                "address": address,
                "error": str(e),
                "analysis_timestamp": datetime.now().isoformat()
            })
    
    return {"batch_results": results}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
doesn't follow real time

import requests
import time
from collections import defaultdict
from typing import Dict, List, Set, Tuple
import json

class EtherscanAnomalyDetector:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.contract_cache = {}
        self.base_url = "https://api.etherscan.io"
        
    # ======================== DATA COLLECTION ========================
    
    def get_transactions(self, address: str, action="txlist", chainid=1, offset=1000, max_pages=5):
        """Fetch transactions with pagination"""
        transactions = []
        for page in range(1, max_pages + 1):
            url = (
                f"{self.base_url}/v2/api"
                f"?chainid={chainid}"
                f"&module=account"
                f"&action={action}"
                f"&address={address}"
                f"&startblock=0"
                f"&endblock=latest"
                f"&page={page}"
                f"&offset={offset}"
                f"&sort=desc"
                f"&apikey={self.api_key}"
            )
            
            response = requests.get(url).json()
            txs = response.get("result", [])
            
            if not txs or txs == "No transactions found":
                break
                
            transactions.extend(txs)
            print(f"Fetched page {page}, got {len(txs)} {action} transactions")
            time.sleep(0.2)  # Rate limiting
            
        return transactions
    
    def is_contract(self, address: str) -> bool:
        """Check if address is contract with caching"""
        if address in self.contract_cache:
            return self.contract_cache[address]
        
        url = f"{self.base_url}/api?module=proxy&action=eth_getCode&address={address}&tag=latest&apikey={self.api_key}"
        response = requests.get(url).json()
        result = response.get("result", "0x") != "0x"
        
        self.contract_cache[address] = result
        return result
    
    def enrich_normal_transactions(self, txs: List[Dict]) -> List[Dict]:
        """Enrich normal transactions with address types"""
        enriched = []
        for tx in txs:
            from_addr = tx["from"]
            to_addr = tx["to"]
            
            tx["from_type"] = "Contract" if self.is_contract(from_addr) else "EOA"
            tx["to_type"] = "Contract" if self.is_contract(to_addr) else "EOA"
            
            enriched.append(tx)
        return enriched
    
    def enrich_internal_transactions(self, txs: List[Dict]) -> List[Dict]:
        """Enrich internal transactions with address types"""
        enriched = []
        for tx in txs:
            from_addr = tx.get("from")
            to_addr = tx.get("to")
            
            if from_addr:
                tx["from_type"] = "Contract" if self.is_contract(from_addr) else "EOA"
            if to_addr:
                tx["to_type"] = "Contract" if self.is_contract(to_addr) else "EOA"
            
            enriched.append(tx)
        return enriched
    
    def enrich_token_transactions(self, txs: List[Dict]) -> List[Dict]:
        """Enrich token transactions with address types and scaled values"""
        # First, classify all unique addresses at once for efficiency
        unique_addresses = set()
        for tx in txs:
            unique_addresses.add(tx["from"])
            unique_addresses.add(tx["to"])
        
        labels = {}
        for addr in unique_addresses:
            labels[addr] = "Contract" if self.is_contract(addr) else "EOA"
            time.sleep(0.1)  # Rate limiting
        
        # Now enrich transactions
        enriched = []
        for tx in txs:
            tx["from_type"] = labels.get(tx["from"], "Unknown")
            tx["to_type"] = labels.get(tx["to"], "Unknown")
            
            # Scale token values
            decimals = int(tx.get("tokenDecimal", 0))
            tx["value_scaled"] = int(tx["value"]) / (10 ** decimals) if decimals else int(tx["value"])
            
            enriched.append(tx)
        
        return enriched
    
    # ======================== ANOMALY DETECTION ========================
    
    def analyze_transaction_frequency(self, txs: List[Dict], window_minutes=60) -> Dict:
        """Detect transaction frequency anomalies"""
        freq_patterns = {}
        for tx in txs:
            timestamp = int(tx.get('timeStamp', 0))
            window_key = timestamp // (window_minutes * 60)
            freq_patterns[window_key] = freq_patterns.get(window_key, 0) + 1
        
        if not freq_patterns:
            return {"anomalies": [], "avg_frequency": 0, "spike_count": 0}
        
        avg_freq = sum(freq_patterns.values()) / len(freq_patterns)
        anomalies = [k for k, v in freq_patterns.items() if v > avg_freq * 5]
        
        return {
            "anomalies": anomalies,
            "avg_frequency": avg_freq,
            "spike_count": len(anomalies),
            "max_frequency": max(freq_patterns.values()) if freq_patterns else 0
        }
    
    def detect_bot_activity(self, txs: List[Dict]) -> Dict:
        """Detect bot-like regular interval patterns"""
        timestamps = [int(tx.get('timeStamp', 0)) for tx in txs]
        timestamps.sort()
        
        if len(timestamps) < 3:
            return {"bot_suspected": False, "regular_interval_count": 0}
        
        intervals = [timestamps[i+1] - timestamps[i] for i in range(len(timestamps)-1)]
        
        # Bot patterns: very regular intervals (±2 seconds)
        regular_intervals = [i for i in intervals if 8 <= i <= 12]  # 10-second bot pattern
        
        return {
            "bot_suspected": len(regular_intervals) > 10,
            "regular_interval_count": len(regular_intervals),
            "total_intervals": len(intervals),
            "regularity_ratio": len(regular_intervals) / len(intervals) if intervals else 0
        }
    
    def analyze_value_anomalies(self, txs: List[Dict]) -> Dict:
        """Detect unusual value patterns - works with both normal and token transactions"""
        values = []
        
        for tx in txs:
            # For token transactions, use scaled values
            if 'value_scaled' in tx and tx['value_scaled'] > 0:
                values.append(float(tx['value_scaled']))
            # For normal transactions, use raw values
            elif 'value' in tx and int(tx['value']) > 0:
                values.append(int(tx['value']))
        
        if not values:
            return {"round_number_ratio": 0, "dust_attack_count": 0, "value_concentration": 0}
        
        avg_value = sum(values) / len(values)
        
        # Detect round number bias (money laundering indicator)
        # For token txs: check if scaled value is round (1.0, 10.0, 100.0)
        # For ETH txs: check if value is exact ETH amounts
        round_values = []
        dust_txs = []
        
        for tx, value in zip(txs, values):
            if 'value_scaled' in tx:  # Token transaction
                if value > 0 and (value == int(value) or value in [0.1, 1.0, 10.0, 100.0, 1000.0]):
                    round_values.append(value)
                if value < 0.001:  # Dust threshold for tokens
                    dust_txs.append(value)
            else:  # ETH transaction
                if value % (10**18) == 0:  # Exact ETH amounts
                    round_values.append(value)
                if value < 10**15:  # < 0.001 ETH
                    dust_txs.append(value)
        
        return {
            "round_number_ratio": len(round_values) / len(values) if values else 0,
            "dust_attack_count": len(dust_txs),
            "value_concentration": max(values) / avg_value if avg_value > 0 else 0,
            "total_value_transfers": len(values)
        }
    
    def analyze_gas_patterns(self, txs: List[Dict]) -> Dict:
        """Analyze gas usage patterns - only for normal transactions"""
        gas_prices = [int(tx.get('gasPrice', 0)) for tx in txs if tx.get('gasPrice')]
        gas_used = [int(tx.get('gasUsed', 0)) for tx in txs if tx.get('gasUsed')]
        
        if not gas_prices:
            return {"high_gas_count": 0, "failed_tx_ratio": 0, "gas_price_variance": 0}
        
        avg_gas_price = sum(gas_prices) / len(gas_prices)
        
        # Detect MEV/sandwich attacks: unusually high gas prices
        high_gas_txs = [g for g in gas_prices if g > avg_gas_price * 3]
        
        # Detect failed transaction spams
        failed_txs = [tx for tx in txs if tx.get('isError') == '1']
        
        return {
            "high_gas_count": len(high_gas_txs),
            "failed_tx_ratio": len(failed_txs) / len(txs) if txs else 0,
            "gas_price_variance": max(gas_prices) / min(gas_prices) if gas_prices and min(gas_prices) > 0 else 0,
            "avg_gas_price": avg_gas_price,
            "total_gas_used": sum(gas_used)
        }
    
    def analyze_address_relationships(self, normal_txs: List[Dict], internal_txs: List[Dict], token_txs: List[Dict]) -> Dict:
        """Analyze interaction patterns between addresses across all transaction types"""
        all_addresses = set()
        interactions = {}
        
        # Build interaction graph from all transaction types
        for tx_list, tx_type in [(normal_txs, 'normal'), (internal_txs, 'internal'), (token_txs, 'token')]:
            for tx in tx_list:
                from_addr = tx.get('from', '')
                to_addr = tx.get('to', '')
                
                if from_addr and to_addr:
                    key = (from_addr, to_addr)
                    if key not in interactions:
                        interactions[key] = {'count': 0, 'types': set(), 'values': []}
                    
                    interactions[key]['count'] += 1
                    interactions[key]['types'].add(tx_type)
                    
                    # Add value information
                    if tx_type == 'token' and 'value_scaled' in tx:
                        interactions[key]['values'].append(tx['value_scaled'])
                    elif tx_type == 'normal' and int(tx.get('value', 0)) > 0:
                        interactions[key]['values'].append(int(tx['value']))
                    
                    all_addresses.update([from_addr, to_addr])
        
        # Detect suspicious patterns
        circular_flows = self.detect_circular_transactions(interactions)
        high_interaction_pairs = [(k, v['count']) for k, v in interactions.items() if v['count'] > 50]
        multi_type_interactions = [k for k, v in interactions.items() if len(v['types']) > 1]
        
        return {
            "unique_addresses": len(all_addresses),
            "interaction_pairs": len(interactions),
            "circular_flows": len(circular_flows),
            "high_interaction_pairs": len(high_interaction_pairs),
            "multi_type_interactions": len(multi_type_interactions),
            "avg_interactions_per_pair": sum(v['count'] for v in interactions.values()) / len(interactions) if interactions else 0
        }
    
    def detect_circular_transactions(self, interactions: Dict) -> List[Dict]:
        """Detect circular transaction patterns (A->B->A)"""
        circular = []
        
        # Build reverse lookup for efficiency
        reverse_interactions = {}
        for (from_addr, to_addr), data in interactions.items():
            if to_addr not in reverse_interactions:
                reverse_interactions[to_addr] = []
            reverse_interactions[to_addr].append((from_addr, data))
        
        # Look for A->B and B->A patterns
        for (from_a, to_b), data_ab in interactions.items():
            if to_b in reverse_interactions:
                for from_b2, data_ba in reverse_interactions[to_b]:
                    if from_b2 == from_a:  # Found A->B->A pattern
                        circular.append({
                            "address_a": from_a,
                            "address_b": to_b,
                            "ab_count": data_ab['count'],
                            "ba_count": data_ba['count'],
                            "ab_types": list(data_ab['types']),
                            "ba_types": list(data_ba['types'])
                        })
        
        return circular
    
    def detect_coordinated_patterns(self, normal_txs: List[Dict], internal_txs: List[Dict], token_txs: List[Dict]) -> Dict:
        """Detect coordinated activity across transaction types"""
        # Group by timestamp
        timestamp_activity = defaultdict(lambda: {'types': set(), 'count': 0})
        
        for tx_list, tx_type in [(normal_txs, 'normal'), (internal_txs, 'internal'), (token_txs, 'token')]:
            for tx in tx_list:
                timestamp = tx.get('timeStamp', '')
                if timestamp:
                    timestamp_activity[timestamp]['types'].add(tx_type)
                    timestamp_activity[timestamp]['count'] += 1
        
        # Find coordinated timestamps (multiple transaction types at same time)
        coordinated_times = []
        for timestamp, activity in timestamp_activity.items():
            if len(activity['types']) >= 2:
                coordinated_times.append({
                    "timestamp": timestamp,
                    "types": list(activity['types']),
                    "transaction_count": activity['count']
                })
        
        return {
            "coordinated_timestamp_count": len(coordinated_times),
            "coordination_ratio": len(coordinated_times) / len(timestamp_activity) if timestamp_activity else 0,
            "total_unique_timestamps": len(timestamp_activity),
            "coordinated_events": coordinated_times[:10]  # Sample
        }
    
    # ======================== SCORING SYSTEM ========================
    
    def calculate_anomaly_score(self, address: str, analysis_results: Dict) -> Dict:
        """Calculate comprehensive anomaly score"""
        score = 0
        flags = []
        
        # Bot activity (30 points)
        if analysis_results.get('bot_patterns', {}).get('bot_suspected', False):
            score += 30
            flags.append("Bot Activity Detected")
        
        # High regularity ratio (additional 15 points)
        regularity = analysis_results.get('bot_patterns', {}).get('regularity_ratio', 0)
        if regularity > 0.7:
            score += 15
            flags.append("High Transaction Regularity")
        
        # Round number bias (20 points)
        round_ratio = analysis_results.get('value_patterns', {}).get('round_number_ratio', 0)
        if round_ratio > 0.5:
            score += 20
            flags.append("Round Number Bias")
        
        # Dust attack detection (15 points)
        dust_count = analysis_results.get('value_patterns', {}).get('dust_attack_count', 0)
        if dust_count > 100:
            score += 15
            flags.append("Potential Dust Attack")
        
        # Failed transaction spam (25 points)
        failed_ratio = analysis_results.get('gas_patterns', {}).get('failed_tx_ratio', 0)
        if failed_ratio > 0.2:
            score += 25
            flags.append("High Failed Transaction Rate")
        
        # Coordinated activity (35 points)
        coord_ratio = analysis_results.get('coordination_patterns', {}).get('coordination_ratio', 0)
        if coord_ratio > 0.3:
            score += 35
            flags.append("Coordinated Multi-Type Transactions")
        
        # Transaction frequency spikes (20 points)
        spike_count = analysis_results.get('frequency_patterns', {}).get('spike_count', 0)
        if spike_count > 3:
            score += 20
            flags.append("Transaction Frequency Spikes")
        
        # Circular transaction patterns (40 points)
        circular_count = analysis_results.get('relationship_patterns', {}).get('circular_flows', 0)
        if circular_count > 0:
            score += 40
            flags.append("Circular Transaction Patterns")
        
        # High gas variance - MEV activity (25 points)
        gas_variance = analysis_results.get('gas_patterns', {}).get('gas_price_variance', 0)
        if gas_variance > 10:
            score += 25
            flags.append("Unusual Gas Price Patterns")
        
        # High interaction concentration (20 points)
        high_interactions = analysis_results.get('relationship_patterns', {}).get('high_interaction_pairs', 0)
        if high_interactions > 5:
            score += 20
            flags.append("High Address Interaction Concentration")
        
        return {
            "address": address,
            "anomaly_score": min(score, 100),
            "risk_level": "HIGH" if score > 70 else "MEDIUM" if score > 40 else "LOW",
            "flags": flags,
            "total_flags": len(flags)
        }
    
    # ======================== MAIN ANALYSIS FUNCTIONS ========================
    
    def analyze_address_comprehensive(self, address: str, max_pages=5) -> Dict:
        """
        Comprehensive analysis strategy:
        1. Fetch all three transaction types
        2. Enrich each type separately 
        3. Analyze patterns within each type
        4. Analyze cross-type relationships
        5. Calculate final anomaly score
        """
        print(f"\n=== Starting Comprehensive Analysis for {address} ===")
        
        # Step 1: Data Collection
        print("📡 Fetching transaction data...")
        normal_txs = self.get_transactions(address, "txlist", max_pages=max_pages)
        internal_txs = self.get_transactions(address, "txlistinternal", max_pages=max_pages) 
        token_txs = self.get_transactions(address, "tokentx", max_pages=max_pages)
        
        print(f"✅ Data collected - Normal: {len(normal_txs)}, Internal: {len(internal_txs)}, Token: {len(token_txs)}")
        
        # Step 2: Data Enrichment
        print("🔍 Enriching transaction data...")
        enriched_normal = self.enrich_normal_transactions(normal_txs)
        enriched_internal = self.enrich_internal_transactions(internal_txs)
        enriched_token = self.enrich_token_transactions(token_txs)
        
        # Step 3: Individual Pattern Analysis
        print("🧠 Analyzing behavioral patterns...")
        
        # Combine all transactions for temporal analysis
        all_txs = enriched_normal + enriched_internal + enriched_token
        
        analysis_results = {
            # Temporal patterns (all transactions)
            "frequency_patterns": self.analyze_transaction_frequency(all_txs),
            "bot_patterns": self.detect_bot_activity(all_txs),
            
            # Value patterns (normal + token transactions with values)
            "value_patterns": self.analyze_value_anomalies(
                [tx for tx in enriched_normal + enriched_token if 
                 ('value_scaled' in tx and tx['value_scaled'] > 0) or 
                 ('value' in tx and int(tx.get('value', 0)) > 0)]
            ),
            
            # Gas patterns (normal transactions only)
            "gas_patterns": self.analyze_gas_patterns(enriched_normal),
            
            # Cross-transaction relationships
            "relationship_patterns": self.analyze_address_relationships(
                enriched_normal, enriched_internal, enriched_token
            ),
            
            # Coordination patterns
            "coordination_patterns": self.detect_coordinated_patterns(
                enriched_normal, enriched_internal, enriched_token
            )
        }
        
        # Step 4: Calculate Final Score
        print("📊 Calculating anomaly score...")
        anomaly_result = self.calculate_anomaly_score(address, analysis_results)
        
        # Step 5: Compile Results
        final_results = {
            "address": address,
            "transaction_summary": {
                "normal_count": len(enriched_normal),
                "internal_count": len(enriched_internal), 
                "token_count": len(enriched_token),
                "total_count": len(all_txs)
            },
            "anomaly_assessment": anomaly_result,
            "detailed_analysis": analysis_results,
            "sample_transactions": {
                "normal": enriched_normal[:3],
                "internal": enriched_internal[:3],
                "token": enriched_token[:3]
            }
        }
        
        print(f"🎯 Analysis Complete! Risk Level: {anomaly_result['risk_level']} (Score: {anomaly_result['anomaly_score']}/100)")
        print(f"🚩 Flags detected: {', '.join(anomaly_result['flags']) if anomaly_result['flags'] else 'None'}")
        
        return final_results

# ======================== USAGE EXAMPLES ========================

if __name__ == "__main__":
    # Initialize detector
    API_KEY = "YOUR_ETHERSCAN_API_KEY"  # Replace with your actual API key
    detector = EtherscanAnomalyDetector(API_KEY)
    
    # Example 1: Analyze Uniswap V3 Router
    uniswap_results = detector.analyze_address_comprehensive(
        "0xE592427A0AEce92De3Edee1F18E0157C05861564",
        max_pages=2  # Reduce for testing
    )
    
    # Print key results
    print("\n" + "="*60)
    print("ANOMALY DETECTION RESULTS")
    print("="*60)
    print(f"Address: {uniswap_results['address']}")
    print(f"Risk Level: {uniswap_results['anomaly_assessment']['risk_level']}")
    print(f"Anomaly Score: {uniswap_results['anomaly_assessment']['anomaly_score']}/100")
    print(f"Flags: {uniswap_results['anomaly_assessment']['flags']}")
    
    # Save results to file
    with open(f"anomaly_results_{uniswap_results['address']}.json", "w") as f:
        json.dump(uniswap_results, f, indent=2, default=str)
    
    print(f"\n💾 Results saved to anomaly_results_{uniswap_results['address']}.json")

In [ ]:
real time

import requests
import time
import asyncio
import threading
from datetime import datetime, timedelta
from collections import defaultdict, deque
from typing import Dict, List, Set, Callable
import json

class RealTimeEtherscanAnomalyDetector:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.contract_cache = {}
        self.base_url = "https://api.etherscan.io"
        
        # Real-time tracking state
        self.monitored_addresses = {}  # {address: last_processed_block}
        self.anomaly_callbacks = []    # List of callback functions for anomalies
        self.is_monitoring = False
        self.monitoring_thread = None
        
        # Sliding window data for real-time analysis
        self.transaction_windows = defaultdict(lambda: {
            'normal': deque(maxlen=1000),
            'internal': deque(maxlen=1000), 
            'token': deque(maxlen=1000),
            'timestamps': deque(maxlen=1000)
        })
        
        # Real-time anomaly thresholds
        self.realtime_thresholds = {
            'frequency_spike_multiplier': 5,
            'bot_interval_tolerance': 2,  # ±2 seconds
            'gas_spike_multiplier': 3,
            'coordination_window': 60,    # seconds
            'min_txs_for_analysis': 10
        }
    
    # ======================== REAL-TIME DATA FETCHING ========================
    
    def get_latest_block(self) -> int:
        """Get the latest block number"""
        url = f"{self.base_url}/api?module=proxy&action=eth_blockNumber&apikey={self.api_key}"
        try:
            response = requests.get(url).json()
            return int(response.get('result', '0x0'), 16)
        except:
            return 0
    
    def get_transactions_from_block(self, address: str, start_block: int, action: str = "txlist") -> List[Dict]:
        """Fetch transactions from a specific block onwards"""
        url = (
            f"{self.base_url}/v2/api"
            f"?chainid=1"
            f"&module=account"
            f"&action={action}"
            f"&address={address}"
            f"&startblock={start_block}"
            f"&endblock=latest"
            f"&page=1"
            f"&offset=100"  # Smaller batches for real-time
            f"&sort=asc"    # Ascending to get newest first
            f"&apikey={self.api_key}"
        )
        
        try:
            response = requests.get(url).json()
            txs = response.get("result", [])
            return txs if isinstance(txs, list) else []
        except Exception as e:
            print(f"Error fetching {action} from block {start_block}: {e}")
            return []
    
    def is_contract(self, address: str) -> bool:
        """Check if address is contract with caching"""
        if address in self.contract_cache:
            return self.contract_cache[address]
        
        url = f"{self.base_url}/api?module=proxy&action=eth_getCode&address={address}&tag=latest&apikey={self.api_key}"
        try:
            response = requests.get(url).json()
            result = response.get("result", "0x") != "0x"
            self.contract_cache[address] = result
            return result
        except:
            return False
    
    # ======================== REAL-TIME ANOMALY DETECTION ========================
    
    def detect_realtime_frequency_anomaly(self, address: str, new_txs: List[Dict]) -> Dict:
        """Detect frequency anomalies in real-time"""
        if not new_txs:
            return {"anomaly": False}
        
        window = self.transaction_windows[address]
        current_time = int(time.time())
        
        # Count transactions in last 5 minutes
        recent_count = 0
        for tx_time in window['timestamps']:
            if current_time - tx_time <= 300:  # 5 minutes
                recent_count += 1
        
        new_tx_count = len(new_txs)
        
        # Anomaly: More than 20 transactions in 5 minutes
        if recent_count + new_tx_count > 20:
            return {
                "anomaly": True,
                "type": "frequency_spike",
                "recent_count": recent_count,
                "new_count": new_tx_count,
                "severity": "HIGH" if recent_count + new_tx_count > 50 else "MEDIUM"
            }
        
        return {"anomaly": False}
    
    def detect_realtime_bot_pattern(self, address: str, new_txs: List[Dict]) -> Dict:
        """Detect bot patterns in real-time"""
        if len(new_txs) < 3:
            return {"anomaly": False}
        
        # Get recent timestamps including new ones
        window = self.transaction_windows[address]
        all_timestamps = list(window['timestamps']) + [int(tx.get('timeStamp', 0)) for tx in new_txs]
        all_timestamps.sort()
        
        if len(all_timestamps) < 10:
            return {"anomaly": False}
        
        # Check last 10 transactions for regular intervals
        recent_timestamps = all_timestamps[-10:]
        intervals = [recent_timestamps[i+1] - recent_timestamps[i] for i in range(len(recent_timestamps)-1)]
        
        # Bot pattern: 7+ out of 9 intervals are within 8-12 seconds
        regular_intervals = [i for i in intervals if 8 <= i <= 12]
        
        if len(regular_intervals) >= 7:
            return {
                "anomaly": True,
                "type": "bot_pattern",
                "regular_intervals": len(regular_intervals),
                "total_intervals": len(intervals),
                "severity": "HIGH"
            }
        
        return {"anomaly": False}
    
    def detect_realtime_gas_anomaly(self, address: str, new_txs: List[Dict]) -> Dict:
        """Detect gas anomalies in real-time - only for normal transactions"""
        normal_txs = [tx for tx in new_txs if tx.get('gasPrice')]
        if not normal_txs:
            return {"anomaly": False}
        
        gas_prices = [int(tx['gasPrice']) for tx in normal_txs]
        
        # Get baseline from recent transactions
        window = self.transaction_windows[address]
        recent_normal = [tx for tx in window['normal'] if tx.get('gasPrice')]
        
        if recent_normal:
            baseline_gas = sum(int(tx['gasPrice']) for tx in recent_normal[-50:]) / min(len(recent_normal), 50)
            
            # Anomaly: New transaction gas > 5x baseline
            high_gas_txs = [g for g in gas_prices if g > baseline_gas * 5]
            
            if high_gas_txs:
                return {
                    "anomaly": True,
                    "type": "gas_spike",
                    "high_gas_count": len(high_gas_txs),
                    "max_gas": max(high_gas_txs),
                    "baseline_gas": baseline_gas,
                    "severity": "HIGH" if max(high_gas_txs) > baseline_gas * 10 else "MEDIUM"
                }
        
        return {"anomaly": False}
    
    def detect_realtime_coordination(self, address: str, normal_txs: List[Dict], 
                                   internal_txs: List[Dict], token_txs: List[Dict]) -> Dict:
        """Detect coordinated activity across transaction types"""
        if not any([normal_txs, internal_txs, token_txs]):
            return {"anomaly": False}
        
        # Check for transactions happening within 60 seconds of each other
        all_new_txs = []
        for tx_list, tx_type in [(normal_txs, 'normal'), (internal_txs, 'internal'), (token_txs, 'token')]:
            for tx in tx_list:
                all_new_txs.append((int(tx.get('timeStamp', 0)), tx_type))
        
        if len(all_new_txs) < 2:
            return {"anomaly": False}
        
        all_new_txs.sort()  # Sort by timestamp
        
        # Look for multiple transaction types within coordination window
        coordinated_groups = []
        window_start = 0
        
        for i in range(len(all_new_txs)):
            current_time = all_new_txs[i][0]
            
            # Find all transactions within window
            window_txs = []
            for j in range(i, len(all_new_txs)):
                if all_new_txs[j][0] - current_time <= self.realtime_thresholds['coordination_window']:
                    window_txs.append(all_new_txs[j])
                else:
                    break
            
            # Check if window has multiple transaction types
            types_in_window = set(tx[1] for tx in window_txs)
            if len(types_in_window) >= 2 and len(window_txs) >= 3:
                coordinated_groups.append({
                    "start_time": current_time,
                    "types": list(types_in_window),
                    "transaction_count": len(window_txs)
                })
        
        if coordinated_groups:
            return {
                "anomaly": True,
                "type": "coordination",
                "coordinated_groups": len(coordinated_groups),
                "severity": "HIGH" if len(coordinated_groups) > 2 else "MEDIUM"
            }
        
        return {"anomaly": False}
    
    # ======================== MONITORING SYSTEM ========================
    
    def add_address_to_monitor(self, address: str, start_block: int = None):
        """Add an address to real-time monitoring"""
        if start_block is None:
            start_block = self.get_latest_block()
        
        self.monitored_addresses[address] = start_block
        print(f"📍 Added {address} to monitoring (starting from block {start_block})")
    
    def add_anomaly_callback(self, callback: Callable):
        """Add callback function to be called when anomalies are detected"""
        self.anomaly_callbacks.append(callback)
    
    def process_new_transactions(self, address: str, normal_txs: List[Dict], 
                               internal_txs: List[Dict], token_txs: List[Dict]):
        """Process new transactions and detect anomalies"""
        if not any([normal_txs, internal_txs, token_txs]):
            return
        
        # Update sliding windows
        window = self.transaction_windows[address]
        current_time = int(time.time())
        
        # Add to windows
        for tx in normal_txs:
            window['normal'].append(tx)
            window['timestamps'].append(int(tx.get('timeStamp', current_time)))
        
        for tx in internal_txs:
            window['internal'].append(tx)
            window['timestamps'].append(int(tx.get('timeStamp', current_time)))
        
        for tx in token_txs:
            window['token'].append(tx)
            window['timestamps'].append(int(tx.get('timeStamp', current_time)))
        
        # Run real-time anomaly detection
        all_new_txs = normal_txs + internal_txs + token_txs
        anomalies_detected = []
        
        # Frequency anomaly
        freq_anomaly = self.detect_realtime_frequency_anomaly(address, all_new_txs)
        if freq_anomaly['anomaly']:
            anomalies_detected.append(freq_anomaly)
        
        # Bot pattern
        bot_anomaly = self.detect_realtime_bot_pattern(address, all_new_txs)
        if bot_anomaly['anomaly']:
            anomalies_detected.append(bot_anomaly)
        
        # Gas anomaly
        gas_anomaly = self.detect_realtime_gas_anomaly(address, normal_txs)
        if gas_anomaly['anomaly']:
            anomalies_detected.append(gas_anomaly)
        
        # Coordination anomaly
        coord_anomaly = self.detect_realtime_coordination(address, normal_txs, internal_txs, token_txs)
        if coord_anomaly['anomaly']:
            anomalies_detected.append(coord_anomaly)
        
        # Trigger callbacks if anomalies detected
        if anomalies_detected:
            anomaly_event = {
                "timestamp": datetime.now().isoformat(),
                "address": address,
                "new_transactions": {
                    "normal": len(normal_txs),
                    "internal": len(internal_txs),
                    "token": len(token_txs)
                },
                "anomalies": anomalies_detected
            }
            
            for callback in self.anomaly_callbacks:
                try:
                    callback(anomaly_event)
                except Exception as e:
                    print(f"Error in anomaly callback: {e}")
    
    def monitoring_loop(self):
        """Main monitoring loop - runs in background thread"""
        print("🚀 Starting real-time monitoring...")
        
        while self.is_monitoring:
            for address, last_block in list(self.monitored_addresses.items()):
                try:
                    # Fetch new transactions from last processed block
                    normal_txs = self.get_transactions_from_block(address, last_block + 1, "txlist")
                    internal_txs = self.get_transactions_from_block(address, last_block + 1, "txlistinternal")
                    token_txs = self.get_transactions_from_block(address, last_block + 1, "tokentx")
                    
                    # Process if we have new transactions
                    if any([normal_txs, internal_txs, token_txs]):
                        print(f"🔄 Processing {len(normal_txs + internal_txs + token_txs)} new transactions for {address}")
                        
                        # Enrich transactions
                        for tx in normal_txs + internal_txs + token_txs:
                            if 'from' in tx and tx['from']:
                                tx['from_type'] = "Contract" if self.is_contract(tx['from']) else "EOA"
                            if 'to' in tx and tx['to']:
                                tx['to_type'] = "Contract" if self.is_contract(tx['to']) else "EOA"
                        
                        # Process for anomalies
                        self.process_new_transactions(address, normal_txs, internal_txs, token_txs)
                        
                        # Update last processed block
                        latest_block = self.get_latest_block()
                        self.monitored_addresses[address] = latest_block
                    
                except Exception as e:
                    print(f"Error monitoring {address}: {e}")
                
                # Rate limiting
                time.sleep(1)  # 1 second between addresses
            
            # Wait before next monitoring cycle
            time.sleep(30)  # Check every 30 seconds
    
    def start_monitoring(self):
        """Start real-time monitoring in background thread"""
        if self.is_monitoring:
            print("⚠️ Monitoring already running")
            return
        
        self.is_monitoring = True
        self.monitoring_thread = threading.Thread(target=self.monitoring_loop, daemon=True)
        self.monitoring_thread.start()
        print("✅ Real-time monitoring started")
    
    def stop_monitoring(self):
        """Stop real-time monitoring"""
        self.is_monitoring = False
        if self.monitoring_thread:
            self.monitoring_thread.join()
        print("🛑 Real-time monitoring stopped")
    
    def get_monitoring_status(self) -> Dict:
        """Get current monitoring status"""
        return {
            "is_monitoring": self.is_monitoring,
            "monitored_addresses": len(self.monitored_addresses),
            "addresses": list(self.monitored_addresses.keys()),
            "callback_count": len(self.anomaly_callbacks)
        }

# ======================== CALLBACK EXAMPLES ========================

def alert_callback(anomaly_event):
    """Example callback that prints alerts"""
    address = anomaly_event['address']
    anomalies = anomaly_event['anomalies']
    
    print(f"\n🚨 ANOMALY ALERT for {address}")
    print(f"Time: {anomaly_event['timestamp']}")
    
    for anomaly in anomalies:
        print(f"  ⚠️ {anomaly['type'].upper()}: {anomaly.get('severity', 'MEDIUM')} severity")
    
    print(f"  📊 New transactions: {anomaly_event['new_transactions']}")

def log_callback(anomaly_event):
    """Example callback that logs to file"""
    with open("realtime_anomalies.log", "a") as f:
        f.write(json.dumps(anomaly_event, indent=2) + "\n")

# ======================== USAGE EXAMPLE ========================

if __name__ == "__main__":
    # Initialize real-time detector
    API_KEY = "YOUR_ETHERSCAN_API_KEY"
    detector = RealTimeEtherscanAnomalyDetector(API_KEY)
    
    # Add callback functions
    detector.add_anomaly_callback(alert_callback)
    detector.add_anomaly_callback(log_callback)
    
    # Add addresses to monitor
    detector.add_address_to_monitor("0xE592427A0AEce92De3Edee1F18E0157C05861564")  # Uniswap V3
    detector.add_address_to_monitor("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D")  # Uniswap V2
    
    # Start monitoring
    detector.start_monitoring()
    
    try:
        # Keep the script running
        while True:
            status = detector.get_monitoring_status()
            print(f"📊 Monitoring {status['monitored_addresses']} addresses...")
            time.sleep(60)  # Print status every minute
    except KeyboardInterrupt:
        print("\n🛑 Stopping monitoring...")
        detector.stop_monitoring()

In [ ]:
include all endpoints

import requests
import time
import asyncio
import threading
from datetime import datetime, timedelta
from collections import defaultdict, deque
from typing import Dict, List, Set, Callable
import json

class RealTimeEtherscanAnomalyDetector:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.contract_cache = {}
        self.base_url = "https://api.etherscan.io"
        
        # Real-time tracking state - separate tracking for each transaction type
        self.monitored_addresses = {}  # {address: {txlist: block, txlistinternal: block, tokentx: block}}
        self.anomaly_callbacks = []    # List of callback functions for anomalies
        self.is_monitoring = False
        self.monitoring_thread = None
        
        # Sliding window data for real-time analysis
        self.transaction_windows = defaultdict(lambda: {
            'normal': deque(maxlen=1000),
            'internal': deque(maxlen=1000), 
            'token': deque(maxlen=1000),
            'timestamps': deque(maxlen=1000)
        })
        
        # Real-time anomaly thresholds
        self.realtime_thresholds = {
            'frequency_spike_multiplier': 5,
            'bot_interval_tolerance': 2,  # ±2 seconds
            'gas_spike_multiplier': 3,
            'coordination_window': 60,    # seconds
            'min_txs_for_analysis': 10
        }
    
    # ======================== REAL-TIME DATA FETCHING ========================
    
    def get_latest_block(self) -> int:
        """Get the latest block number"""
        url = f"{self.base_url}/api?module=proxy&action=eth_blockNumber&apikey={self.api_key}"
        try:
            response = requests.get(url).json()
            return int(response.get('result', '0x0'), 16)
        except:
            return 0
    
    def get_transactions_since_block(self, address: str, start_block: int, action: str) -> List[Dict]:
        """Fetch transactions from a specific block onwards for any transaction type"""
        # Different API endpoints for different transaction types
        if action == "txlist":
            module = "account"
        elif action == "txlistinternal":
            module = "account" 
        elif action == "tokentx":
            module = "account"
        else:
            return []
            
        url = (
            f"{self.base_url}/v2/api"
            f"?chainid=1"
            f"&module={module}"
            f"&action={action}"
            f"&address={address}"
            f"&startblock={start_block}"
            f"&endblock=latest"
            f"&page=1"
            f"&offset=100"  # Smaller batches for real-time
            f"&sort=asc"    # Ascending to get chronological order
            f"&apikey={self.api_key}"
        )
        
        try:
            response = requests.get(url).json()
            txs = response.get("result", [])
            
            # Handle different response formats
            if isinstance(txs, list):
                return txs
            elif txs == "No transactions found":
                return []
            else:
                return []
                
        except Exception as e:
            print(f"Error fetching {action} from block {start_block}: {e}")
            return []
    
    def get_highest_block_from_txs(self, txs: List[Dict]) -> int:
        """Get the highest block number from a list of transactions"""
        if not txs:
            return 0
        
        block_numbers = []
        for tx in txs:
            block_num = tx.get('blockNumber', '0')
            try:
                block_numbers.append(int(block_num))
            except (ValueError, TypeError):
                continue
        
        return max(block_numbers) if block_numbers else 0
    
    def is_contract(self, address: str) -> bool:
        """Check if address is contract with caching"""
        if address in self.contract_cache:
            return self.contract_cache[address]
        
        url = f"{self.base_url}/api?module=proxy&action=eth_getCode&address={address}&tag=latest&apikey={self.api_key}"
        try:
            response = requests.get(url).json()
            result = response.get("result", "0x") != "0x"
            self.contract_cache[address] = result
            return result
        except:
            return False
    
    # ======================== REAL-TIME ANOMALY DETECTION ========================
    
    def detect_realtime_frequency_anomaly(self, address: str, new_txs: List[Dict]) -> Dict:
        """Detect frequency anomalies in real-time"""
        if not new_txs:
            return {"anomaly": False}
        
        window = self.transaction_windows[address]
        current_time = int(time.time())
        
        # Count transactions in last 5 minutes
        recent_count = 0
        for tx_time in window['timestamps']:
            if current_time - tx_time <= 300:  # 5 minutes
                recent_count += 1
        
        new_tx_count = len(new_txs)
        
        # Anomaly: More than 20 transactions in 5 minutes
        if recent_count + new_tx_count > 20:
            return {
                "anomaly": True,
                "type": "frequency_spike",
                "recent_count": recent_count,
                "new_count": new_tx_count,
                "severity": "HIGH" if recent_count + new_tx_count > 50 else "MEDIUM"
            }
        
        return {"anomaly": False}
    
    def detect_realtime_bot_pattern(self, address: str, new_txs: List[Dict]) -> Dict:
        """Detect bot patterns in real-time"""
        if len(new_txs) < 3:
            return {"anomaly": False}
        
        # Get recent timestamps including new ones
        window = self.transaction_windows[address]
        all_timestamps = list(window['timestamps']) + [int(tx.get('timeStamp', 0)) for tx in new_txs]
        all_timestamps.sort()
        
        if len(all_timestamps) < 10:
            return {"anomaly": False}
        
        # Check last 10 transactions for regular intervals
        recent_timestamps = all_timestamps[-10:]
        intervals = [recent_timestamps[i+1] - recent_timestamps[i] for i in range(len(recent_timestamps)-1)]
        
        # Bot pattern: 7+ out of 9 intervals are within 8-12 seconds
        regular_intervals = [i for i in intervals if 8 <= i <= 12]
        
        if len(regular_intervals) >= 7:
            return {
                "anomaly": True,
                "type": "bot_pattern",
                "regular_intervals": len(regular_intervals),
                "total_intervals": len(intervals),
                "severity": "HIGH"
            }
        
        return {"anomaly": False}
    
    def detect_realtime_gas_anomaly(self, address: str, new_txs: List[Dict]) -> Dict:
        """Detect gas anomalies in real-time - only for normal transactions"""
        normal_txs = [tx for tx in new_txs if tx.get('gasPrice')]
        if not normal_txs:
            return {"anomaly": False}
        
        gas_prices = [int(tx['gasPrice']) for tx in normal_txs]
        
        # Get baseline from recent transactions
        window = self.transaction_windows[address]
        recent_normal = [tx for tx in window['normal'] if tx.get('gasPrice')]
        
        if recent_normal:
            baseline_gas = sum(int(tx['gasPrice']) for tx in recent_normal[-50:]) / min(len(recent_normal), 50)
            
            # Anomaly: New transaction gas > 5x baseline
            high_gas_txs = [g for g in gas_prices if g > baseline_gas * 5]
            
            if high_gas_txs:
                return {
                    "anomaly": True,
                    "type": "gas_spike",
                    "high_gas_count": len(high_gas_txs),
                    "max_gas": max(high_gas_txs),
                    "baseline_gas": baseline_gas,
                    "severity": "HIGH" if max(high_gas_txs) > baseline_gas * 10 else "MEDIUM"
                }
        
        return {"anomaly": False}
    
    def detect_realtime_coordination(self, address: str, normal_txs: List[Dict], 
                                   internal_txs: List[Dict], token_txs: List[Dict]) -> Dict:
        """Detect coordinated activity across transaction types"""
        if not any([normal_txs, internal_txs, token_txs]):
            return {"anomaly": False}
        
        # Check for transactions happening within 60 seconds of each other
        all_new_txs = []
        for tx_list, tx_type in [(normal_txs, 'normal'), (internal_txs, 'internal'), (token_txs, 'token')]:
            for tx in tx_list:
                all_new_txs.append((int(tx.get('timeStamp', 0)), tx_type))
        
        if len(all_new_txs) < 2:
            return {"anomaly": False}
        
        all_new_txs.sort()  # Sort by timestamp
        
        # Look for multiple transaction types within coordination window
        coordinated_groups = []
        window_start = 0
        
        for i in range(len(all_new_txs)):
            current_time = all_new_txs[i][0]
            
            # Find all transactions within window
            window_txs = []
            for j in range(i, len(all_new_txs)):
                if all_new_txs[j][0] - current_time <= self.realtime_thresholds['coordination_window']:
                    window_txs.append(all_new_txs[j])
                else:
                    break
            
            # Check if window has multiple transaction types
            types_in_window = set(tx[1] for tx in window_txs)
            if len(types_in_window) >= 2 and len(window_txs) >= 3:
                coordinated_groups.append({
                    "start_time": current_time,
                    "types": list(types_in_window),
                    "transaction_count": len(window_txs)
                })
        
        if coordinated_groups:
            return {
                "anomaly": True,
                "type": "coordination",
                "coordinated_groups": len(coordinated_groups),
                "severity": "HIGH" if len(coordinated_groups) > 2 else "MEDIUM"
            }
        
        return {"anomaly": False}
    
    # ======================== MONITORING SYSTEM ========================
    
    def add_address_to_monitor(self, address: str, start_block: int = None):
        """Add an address to real-time monitoring with separate tracking for each tx type"""
        if start_block is None:
            start_block = self.get_latest_block()
        
        # Initialize tracking for all three transaction types
        self.monitored_addresses[address] = {
            'txlist': start_block,
            'txlistinternal': start_block,
            'tokentx': start_block
        }
        print(f"📍 Added {address} to monitoring (starting from block {start_block})")
        print(f"   Tracking: normal, internal, and token transactions")
    
    def add_anomaly_callback(self, callback: Callable):
        """Add callback function to be called when anomalies are detected"""
        self.anomaly_callbacks.append(callback)
    
    def process_new_transactions(self, address: str, normal_txs: List[Dict], 
                               internal_txs: List[Dict], token_txs: List[Dict]):
        """Process new transactions and detect anomalies"""
        if not any([normal_txs, internal_txs, token_txs]):
            return
        
        # Update sliding windows
        window = self.transaction_windows[address]
        current_time = int(time.time())
        
        # Add to windows
        for tx in normal_txs:
            window['normal'].append(tx)
            window['timestamps'].append(int(tx.get('timeStamp', current_time)))
        
        for tx in internal_txs:
            window['internal'].append(tx)
            window['timestamps'].append(int(tx.get('timeStamp', current_time)))
        
        for tx in token_txs:
            window['token'].append(tx)
            window['timestamps'].append(int(tx.get('timeStamp', current_time)))
        
        # Run real-time anomaly detection
        all_new_txs = normal_txs + internal_txs + token_txs
        anomalies_detected = []
        
        # Frequency anomaly
        freq_anomaly = self.detect_realtime_frequency_anomaly(address, all_new_txs)
        if freq_anomaly['anomaly']:
            anomalies_detected.append(freq_anomaly)
        
        # Bot pattern
        bot_anomaly = self.detect_realtime_bot_pattern(address, all_new_txs)
        if bot_anomaly['anomaly']:
            anomalies_detected.append(bot_anomaly)
        
        # Gas anomaly
        gas_anomaly = self.detect_realtime_gas_anomaly(address, normal_txs)
        if gas_anomaly['anomaly']:
            anomalies_detected.append(gas_anomaly)
        
        # Coordination anomaly
        coord_anomaly = self.detect_realtime_coordination(address, normal_txs, internal_txs, token_txs)
        if coord_anomaly['anomaly']:
            anomalies_detected.append(coord_anomaly)
        
        # Trigger callbacks if anomalies detected
        if anomalies_detected:
            anomaly_event = {
                "timestamp": datetime.now().isoformat(),
                "address": address,
                "new_transactions": {
                    "normal": len(normal_txs),
                    "internal": len(internal_txs),
                    "token": len(token_txs)
                },
                "anomalies": anomalies_detected
            }
            
            for callback in self.anomaly_callbacks:
                try:
                    callback(anomaly_event)
                except Exception as e:
                    print(f"Error in anomaly callback: {e}")
    
    def monitoring_loop(self):
        """Main monitoring loop - runs in background thread"""
        print("🚀 Starting real-time monitoring for all transaction types...")
        
        while self.is_monitoring:
            for address, tracking_info in list(self.monitored_addresses.items()):
                try:
                    new_normal_txs = []
                    new_internal_txs = []
                    new_token_txs = []
                    
                    # Fetch new transactions for each type separately
                    print(f"🔍 Checking {address} for new transactions...")
                    
                    # Normal transactions (txlist)
                    normal_start_block = tracking_info['txlist'] + 1
                    raw_normal = self.get_transactions_since_block(address, normal_start_block, "txlist")
                    if raw_normal:
                        new_normal_txs = self.enrich_transactions_realtime(raw_normal, 'normal')
                        # Update tracking
                        highest_normal_block = self.get_highest_block_from_txs(raw_normal)
                        if highest_normal_block > 0:
                            tracking_info['txlist'] = highest_normal_block
                        print(f"   📥 {len(new_normal_txs)} new normal transactions")
                    
                    # Internal transactions (txlistinternal)  
                    internal_start_block = tracking_info['txlistinternal'] + 1
                    raw_internal = self.get_transactions_since_block(address, internal_start_block, "txlistinternal")
                    if raw_internal:
                        new_internal_txs = self.enrich_transactions_realtime(raw_internal, 'internal')
                        # Update tracking
                        highest_internal_block = self.get_highest_block_from_txs(raw_internal)
                        if highest_internal_block > 0:
                            tracking_info['txlistinternal'] = highest_internal_block
                        print(f"   📥 {len(new_internal_txs)} new internal transactions")
                    
                    # Token transactions (tokentx)
                    token_start_block = tracking_info['tokentx'] + 1
                    raw_token = self.get_transactions_since_block(address, token_start_block, "tokentx")
                    if raw_token:
                        new_token_txs = self.enrich_transactions_realtime(raw_token, 'token')
                        # Update tracking
                        highest_token_block = self.get_highest_block_from_txs(raw_token)
                        if highest_token_block > 0:
                            tracking_info['tokentx'] = highest_token_block
                        print(f"   📥 {len(new_token_txs)} new token transactions")
                    
                    # Process all new transactions if we have any
                    if any([new_normal_txs, new_internal_txs, new_token_txs]):
                        total_new = len(new_normal_txs) + len(new_internal_txs) + len(new_token_txs)
                        print(f"🔄 Processing {total_new} total new transactions for {address}")
                        
                        # Process for anomalies
                        self.process_new_transactions(address, new_normal_txs, new_internal_txs, new_token_txs)
                    else:
                        print(f"   ✅ No new transactions for {address}")
                    
                except Exception as e:
                    print(f"❌ Error monitoring {address}: {e}")
                
                # Rate limiting between addresses
                time.sleep(2)  # 2 seconds between addresses to respect API limits
            
            # Wait before next monitoring cycle
            print(f"⏱️ Monitoring cycle complete. Waiting 45 seconds before next cycle...")
            time.sleep(45)  # Check every 45 seconds
    
    def start_monitoring(self):
        """Start real-time monitoring in background thread"""
        if self.is_monitoring:
            print("⚠️ Monitoring already running")
            return
        
        self.is_monitoring = True
        self.monitoring_thread = threading.Thread(target=self.monitoring_loop, daemon=True)
        self.monitoring_thread.start()
        print("✅ Real-time monitoring started")
    
    def stop_monitoring(self):
        """Stop real-time monitoring"""
        self.is_monitoring = False
        if self.monitoring_thread:
            self.monitoring_thread.join()
        print("🛑 Real-time monitoring stopped")
    
    def get_monitoring_status(self) -> Dict:
        """Get current monitoring status"""
        return {
            "is_monitoring": self.is_monitoring,
            "monitored_addresses": len(self.monitored_addresses),
            "addresses": list(self.monitored_addresses.keys()),
            "callback_count": len(self.anomaly_callbacks)
        }

# ======================== CALLBACK EXAMPLES ========================

def alert_callback(anomaly_event):
    """Example callback that prints alerts"""
    address = anomaly_event['address']
    anomalies = anomaly_event['anomalies']
    
    print(f"\n🚨 ANOMALY ALERT for {address}")
    print(f"Time: {anomaly_event['timestamp']}")
    
    for anomaly in anomalies:
        print(f"  ⚠️ {anomaly['type'].upper()}: {anomaly.get('severity', 'MEDIUM')} severity")
    
    print(f"  📊 New transactions: {anomaly_event['new_transactions']}")

def log_callback(anomaly_event):
    """Example callback that logs to file"""
    with open("realtime_anomalies.log", "a") as f:
        f.write(json.dumps(anomaly_event, indent=2) + "\n")

# ======================== USAGE EXAMPLE ========================

if __name__ == "__main__":
    # Initialize real-time detector
    API_KEY = "YOUR_ETHERSCAN_API_KEY"
    detector = RealTimeEtherscanAnomalyDetector(API_KEY)
    
    # Add callback functions
    detector.add_anomaly_callback(alert_callback)
    detector.add_anomaly_callback(log_callback)
    
    # Add addresses to monitor
    detector.add_address_to_monitor("0xE592427A0AEce92De3Edee1F18E0157C05861564")  # Uniswap V3
    detector.add_address_to_monitor("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D")  # Uniswap V2
    
    # Start monitoring
    detector.start_monitoring()
    
    try:
        # Keep the script running
        while True:
            status = detector.get_monitoring_status()
            print(f"📊 Monitoring {status['monitored_addresses']} addresses...")
            time.sleep(60)  # Print status every minute
    except KeyboardInterrupt:
        print("\n🛑 Stopping monitoring...")
        detector.stop_monitoring()

In [ ]:
import requests
import time
import asyncio
import threading
from datetime import datetime, timedelta
from collections import defaultdict, deque
from typing import Dict, List, Set, Callable, Optional
import json

class ReactiveEtherscanAnomalyDetector:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://api.etherscan.io"
        
        # Caching system
        self.contract_cache = {}  # {address: bool}
        self.address_cache = {}   # {address: {last_check: timestamp, data: recent_txs}}
        self.cache_ttl = 300      # 5 minutes cache TTL
        
        # Rate limiting
        self.last_request_time = 0
        self.min_request_interval = 0.2  # 200ms between requests (5 req/sec max)
        self.daily_request_count = 0
        self.daily_limit = 100000  # Etherscan daily limit
        
        # Reactive monitoring state
        self.anomaly_triggers = []  # List of trigger functions
        self.alert_callbacks = []
        self.monitoring_window_hours = 48  # Focus on last 48 hours
        
        # Quick check system (lightweight monitoring)
        self.quick_check_addresses = set()
        self.is_quick_monitoring = False
        self.quick_monitor_thread = None
        
    # ======================== RATE LIMITING & CACHING ========================
    
    def _rate_limit(self):
        """Enforce rate limiting"""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        
        if time_since_last < self.min_request_interval:
            sleep_time = self.min_request_interval - time_since_last
            time.sleep(sleep_time)
        
        self.last_request_time = time.time()
        self.daily_request_count += 1
        
        if self.daily_request_count >= self.daily_limit:
            raise Exception("Daily API limit reached")
    
    def _get_cache_key(self, address: str, action: str) -> str:
        """Generate cache key for address and action"""
        return f"{address}:{action}"
    
    def _is_cache_valid(self, address: str, action: str) -> bool:
        """Check if cached data is still valid"""
        cache_key = self._get_cache_key(address, action)
        if cache_key not in self.address_cache:
            return False
        
        cached_data = self.address_cache[cache_key]
        cache_age = time.time() - cached_data['last_check']
        return cache_age < self.cache_ttl
    
    def _get_cached_data(self, address: str, action: str) -> Optional[List[Dict]]:
        """Get cached data if valid"""
        if self._is_cache_valid(address, action):
            cache_key = self._get_cache_key(address, action)
            return self.address_cache[cache_key]['data']
        return None
    
    def _cache_data(self, address: str, action: str, data: List[Dict]):
        """Cache transaction data"""
        cache_key = self._get_cache_key(address, action)
        self.address_cache[cache_key] = {
            'last_check': time.time(),
            'data': data
        }
    
    # ======================== REACTIVE DATA FETCHING ========================
    
    def get_recent_transactions(self, address: str, action: str, hours: int = 48) -> List[Dict]:
        """Get recent transactions (last N hours) with caching"""
        # Check cache first
        cached_data = self._get_cached_data(address, action)
        if cached_data is not None:
            print(f"📦 Using cached {action} data for {address}")
            return cached_data
        
        # Calculate start block for recent timeframe
        current_time = int(time.time())
        start_time = current_time - (hours * 3600)  # N hours ago
        
        # Estimate block number (roughly 12 seconds per block)
        blocks_back = hours * 300  # Approximate blocks in N hours
        latest_block = self.get_latest_block()
        start_block = max(0, latest_block - blocks_back)
        
        print(f"🔍 Fetching recent {action} for {address} (last {hours}h, from block {start_block})")
        
        # Fetch data with rate limiting
        self._rate_limit()
        
        url = (
            f"{self.base_url}/v2/api"
            f"?chainid=1"
            f"&module=account"
            f"&action={action}"
            f"&address={address}"
            f"&startblock={start_block}"
            f"&endblock=latest"
            f"&page=1"
            f"&offset=1000"
            f"&sort=desc"
            f"&apikey={self.api_key}"
        )
        
        try:
            response = requests.get(url).json()
            txs = response.get("result", [])
            
            if isinstance(txs, list):
                # Filter to exact time window and cache
                recent_txs = [tx for tx in txs if int(tx.get('timeStamp', 0)) >= start_time]
                self._cache_data(address, action, recent_txs)
                print(f"✅ Fetched {len(recent_txs)} recent {action} transactions")
                return recent_txs
            else:
                print(f"⚠️ No {action} transactions found for {address}")
                self._cache_data(address, action, [])
                return []
                
        except Exception as e:
            print(f"❌ Error fetching {action} for {address}: {e}")
            return []
    
    def get_latest_block(self) -> int:
        """Get latest block number with minimal API usage"""
        self._rate_limit()
        url = f"{self.base_url}/api?module=proxy&action=eth_blockNumber&apikey={self.api_key}"
        try:
            response = requests.get(url).json()
            return int(response.get('result', '0x0'), 16)
        except:
            return 0
    
    def is_contract(self, address: str) -> bool:
        """Check if address is contract with caching"""
        if address in self.contract_cache:
            return self.contract_cache[address]
        
        self._rate_limit()
        url = f"{self.base_url}/api?module=proxy&action=eth_getCode&address={address}&tag=latest&apikey={self.api_key}"
        
        try:
            response = requests.get(url).json()
            result = response.get("result", "0x") != "0x"
            self.contract_cache[address] = result
            return result
        except:
            return False
    
    # ======================== LIGHTWEIGHT TRIGGERS ========================
    
    def quick_transaction_count_check(self, address: str) -> Dict:
        """Lightweight check - just count recent transactions without full analysis"""
        try:
            # Very recent check - last 1 hour only
            recent_normal = len(self.get_recent_transactions(address, "txlist", hours=1))
            recent_internal = len(self.get_recent_transactions(address, "txlistinternal", hours=1))
            recent_token = len(self.get_recent_transactions(address, "tokentx", hours=1))
            
            total_recent = recent_normal + recent_internal + recent_token
            
            return {
                "address": address,
                "total_1h": total_recent,
                "normal_1h": recent_normal,
                "internal_1h": recent_internal,
                "token_1h": recent_token,
                "high_activity": total_recent > 50,  # Trigger threshold
                "timestamp": datetime.now().isoformat()
            }
            
        except Exception as e:
            print(f"Error in quick check for {address}: {e}")
            return {"address": address, "high_activity": False}
    
    def add_anomaly_trigger(self, trigger_func: Callable) -> None:
        """Add a custom trigger function"""
        self.anomaly_triggers.append(trigger_func)
    
    def add_alert_callback(self, callback: Callable) -> None:
        """Add callback for anomaly alerts"""
        self.alert_callbacks.append(callback)
    
    # ======================== REACTIVE ANOMALY ANALYSIS ========================
    
    def analyze_on_trigger(self, address: str, trigger_reason: str) -> Dict:
        """Perform detailed analysis ONLY when triggered"""
        print(f"\n🎯 TRIGGERED ANALYSIS for {address}")
        print(f"📋 Reason: {trigger_reason}")
        
        # Now fetch detailed data reactively
        print("📡 Fetching comprehensive recent data...")
        
        normal_txs = self.get_recent_transactions(address, "txlist", self.monitoring_window_hours)
        internal_txs = self.get_recent_transactions(address, "txlistinternal", self.monitoring_window_hours)
        token_txs = self.get_recent_transactions(address, "tokentx", self.monitoring_window_hours)
        
        # Enrich transactions
        enriched_normal = self._enrich_transactions(normal_txs, 'normal')
        enriched_internal = self._enrich_transactions(internal_txs, 'internal')
        enriched_token = self._enrich_transactions(token_txs, 'token')
        
        # Perform detailed anomaly analysis
        analysis_results = {
            "trigger_reason": trigger_reason,
            "data_window": f"Last {self.monitoring_window_hours} hours",
            "transaction_counts": {
                "normal": len(enriched_normal),
                "internal": len(enriched_internal),
                "token": len(enriched_token),
                "total": len(enriched_normal) + len(enriched_internal) + len(enriched_token)
            }
        }
        
        # Only analyze if we have sufficient data
        if analysis_results["transaction_counts"]["total"] >= 10:
            all_txs = enriched_normal + enriched_internal + enriched_token
            
            analysis_results.update({
                "frequency_analysis": self._analyze_frequency_patterns(all_txs),
                "bot_analysis": self._analyze_bot_patterns(all_txs),
                "value_analysis": self._analyze_value_patterns(enriched_normal + enriched_token),
                "gas_analysis": self._analyze_gas_patterns(enriched_normal),
                "coordination_analysis": self._analyze_coordination(enriched_normal, enriched_internal, enriched_token)
            })
            
            # Calculate risk score
            risk_assessment = self._calculate_risk_score(analysis_results)
            analysis_results["risk_assessment"] = risk_assessment
            
            # Trigger alerts if high risk
            if risk_assessment["risk_level"] in ["HIGH", "CRITICAL"]:
                self._trigger_alerts(address, analysis_results)
        
        else:
            analysis_results["insufficient_data"] = True
            print("⚠️ Insufficient recent transaction data for detailed analysis")
        
        return analysis_results
    
    def _enrich_transactions(self, txs: List[Dict], tx_type: str) -> List[Dict]:
        """Enrich transactions with address types and scaled values"""
        if not txs:
            return []
        
        enriched = []
        for tx in txs:
            tx['tx_type'] = tx_type
            
            # Add address types (batch check for efficiency)
            if tx.get('from'):
                tx['from_type'] = "Contract" if self.is_contract(tx['from']) else "EOA"
            if tx.get('to'):
                tx['to_type'] = "Contract" if self.is_contract(tx['to']) else "EOA"
            
            # For token transactions, add scaled values
            if tx_type == 'token' and 'tokenDecimal' in tx:
                try:
                    decimals = int(tx.get('tokenDecimal', 0))
                    value = int(tx.get('value', 0))
                    tx['value_scaled'] = value / (10 ** decimals) if decimals else value
                except:
                    tx['value_scaled'] = 0
            
            enriched.append(tx)
        
        return enriched
    
    # ======================== QUICK MONITORING SYSTEM ========================
    
    def start_reactive_monitoring(self, addresses: List[str]):
        """Start lightweight reactive monitoring"""
        self.quick_check_addresses = set(addresses)
        self.is_quick_monitoring = True
        
        def monitoring_loop():
            print(f"🚀 Starting reactive monitoring for {len(addresses)} addresses")
            print(f"📊 Monitoring window: Last {self.monitoring_window_hours} hours")
            
            while self.is_quick_monitoring:
                for address in list(self.quick_check_addresses):
                    try:
                        # Lightweight check
                        quick_result = self.quick_transaction_count_check(address)
                        
                        # Trigger detailed analysis if threshold exceeded
                        if quick_result.get("high_activity", False):
                            print(f"\n⚡ HIGH ACTIVITY DETECTED: {address}")
                            print(f"   📈 {quick_result['total_1h']} transactions in last hour")
                            
                            # Reactive detailed analysis
                            detailed_analysis = self.analyze_on_trigger(
                                address, 
                                f"High activity: {quick_result['total_1h']} txs/hour"
                            )
                        
                        # Custom triggers
                        for trigger_func in self.anomaly_triggers:
                            try:
                                trigger_result = trigger_func(address, quick_result)
                                if trigger_result.get("triggered", False):
                                    self.analyze_on_trigger(address, trigger_result["reason"])
                            except Exception as e:
                                print(f"Error in custom trigger: {e}")
                        
                    except Exception as e:
                        print(f"Error in monitoring {address}: {e}")
                    
                    # Rate limiting between addresses
                    time.sleep(10)  # 10 seconds between address checks
                
                # Wait before next monitoring cycle
                print(f"💤 Monitoring cycle complete. Next check in 5 minutes...")
                time.sleep(300)  # 5 minute cycles for lightweight checks
        
        self.quick_monitor_thread = threading.Thread(target=monitoring_loop, daemon=True)
        self.quick_monitor_thread.start()
    
    def stop_reactive_monitoring(self):
        """Stop reactive monitoring"""
        self.is_quick_monitoring = False
        print("🛑 Reactive monitoring stopped")
    
    # ======================== ANALYSIS METHODS (Simplified) ========================
    
    def _analyze_frequency_patterns(self, txs: List[Dict]) -> Dict:
        """Analyze transaction frequency patterns"""
        if len(txs) < 5:
            return {"insufficient_data": True}
        
        timestamps = [int(tx.get('timeStamp', 0)) for tx in txs]
        timestamps.sort()
        
        # Check for bursts (>10 txs in 10 minutes)
        burst_windows = []
        window_size = 600  # 10 minutes
        
        for i, ts in enumerate(timestamps):
            window_end = ts + window_size
            count_in_window = sum(1 for t in timestamps[i:] if t <= window_end)
            
            if count_in_window > 10:
                burst_windows.append({"start": ts, "count": count_in_window})
        
        return {
            "total_transactions": len(txs),
            "time_span_hours": (max(timestamps) - min(timestamps)) / 3600,
            "burst_windows": len(burst_windows),
            "high_frequency": len(burst_windows) > 0
        }
    
    def _analyze_bot_patterns(self, txs: List[Dict]) -> Dict:
        """Analyze for bot-like patterns"""
        if len(txs) < 10:
            return {"insufficient_data": True}
        
        timestamps = sorted([int(tx.get('timeStamp', 0)) for tx in txs])
        intervals = [timestamps[i+1] - timestamps[i] for i in range(len(timestamps)-1)]
        
        # Look for regular intervals
        regular_10s = sum(1 for i in intervals if 8 <= i <= 12)
        regular_60s = sum(1 for i in intervals if 58 <= i <= 62)
        
        return {
            "regular_10s_intervals": regular_10s,
            "regular_60s_intervals": regular_60s,
            "total_intervals": len(intervals),
            "bot_suspected": (regular_10s > len(intervals) * 0.7) or (regular_60s > len(intervals) * 0.7)
        }
    
    def _analyze_value_patterns(self, txs: List[Dict]) -> Dict:
        """Analyze value transfer patterns"""
        values = []
        for tx in txs:
            if 'value_scaled' in tx and tx['value_scaled'] > 0:
                values.append(tx['value_scaled'])
            elif int(tx.get('value', 0)) > 0:
                values.append(int(tx['value']))
        
        if not values:
            return {"no_value_transfers": True}
        
        # Check for round numbers and suspicious patterns
        round_values = sum(1 for v in values if v == int(v) and v > 0)
        
        return {
            "total_value_transfers": len(values),
            "round_number_ratio": round_values / len(values),
            "suspicious_round_numbers": round_values / len(values) > 0.8
        }
    
    def _analyze_gas_patterns(self, normal_txs: List[Dict]) -> Dict:
        """Analyze gas usage patterns"""
        gas_prices = [int(tx.get('gasPrice', 0)) for tx in normal_txs if tx.get('gasPrice')]
        
        if not gas_prices:
            return {"no_gas_data": True}
        
        avg_gas = sum(gas_prices) / len(gas_prices)
        high_gas_count = sum(1 for g in gas_prices if g > avg_gas * 5)
        
        return {
            "avg_gas_price": avg_gas,
            "high_gas_transactions": high_gas_count,
            "suspicious_gas_usage": high_gas_count > len(gas_prices) * 0.1
        }
    
    def _analyze_coordination(self, normal_txs: List[Dict], internal_txs: List[Dict], token_txs: List[Dict]) -> Dict:
        """Analyze coordination across transaction types"""
        timestamp_types = defaultdict(set)
        
        for tx_list, tx_type in [(normal_txs, 'normal'), (internal_txs, 'internal'), (token_txs, 'token')]:
            for tx in tx_list:
                ts = tx.get('timeStamp', '')
                if ts:
                    timestamp_types[ts].add(tx_type)
        
        coordinated_timestamps = sum(1 for types in timestamp_types.values() if len(types) > 1)
        
        return {
            "coordinated_timestamps": coordinated_timestamps,
            "total_timestamps": len(timestamp_types),
            "coordination_ratio": coordinated_timestamps / len(timestamp_types) if timestamp_types else 0,
            "high_coordination": coordinated_timestamps / len(timestamp_types) > 0.3 if timestamp_types else False
        }
    
    def _calculate_risk_score(self, analysis: Dict) -> Dict:
        """Calculate overall risk score"""
        score = 0
        flags = []
        
        # High frequency activity
        if analysis.get("frequency_analysis", {}).get("high_frequency", False):
            score += 25
            flags.append("High Frequency Bursts")
        
        # Bot patterns
        if analysis.get("bot_analysis", {}).get("bot_suspected", False):
            score += 35
            flags.append("Bot-like Patterns")
        
        # Suspicious values
        if analysis.get("value_analysis", {}).get("suspicious_round_numbers", False):
            score += 20
            flags.append("Suspicious Value Patterns")
        
        # Gas manipulation
        if analysis.get("gas_analysis", {}).get("suspicious_gas_usage", False):
            score += 30
            flags.append("Gas Price Manipulation")
        
        # Coordination
        if analysis.get("coordination_analysis", {}).get("high_coordination", False):
            score += 40
            flags.append("Multi-Type Coordination")
        
        # Risk levels
        if score >= 80:
            risk_level = "CRITICAL"
        elif score >= 60:
            risk_level = "HIGH"
        elif score >= 30:
            risk_level = "MEDIUM"
        else:
            risk_level = "LOW"
        
        return {
            "risk_score": min(score, 100),
            "risk_level": risk_level,
            "flags": flags
        }
    
    def _trigger_alerts(self, address: str, analysis: Dict):
        """Trigger alert callbacks"""
        alert_data = {
            "timestamp": datetime.now().isoformat(),
            "address": address,
            "analysis": analysis
        }
        
        for callback in self.alert_callbacks:
            try:
                callback(alert_data)
            except Exception as e:
                print(f"Error in alert callback: {e}")

# ======================== USAGE EXAMPLE ========================

def reactive_alert_callback(alert_data):
    """Callback for reactive alerts"""
    address = alert_data['address']
    analysis = alert_data['analysis']
    risk = analysis['risk_assessment']
    
    print(f"\n🚨 REACTIVE ANOMALY DETECTED!")
    print(f"📍 Address: {address}")
    print(f"⏰ Time: {alert_data['timestamp']}")
    print(f"🎯 Trigger: {analysis['trigger_reason']}")
    print(f"📊 Risk Level: {risk['risk_level']} ({risk['risk_score']}/100)")
    print(f"🚩 Flags: {', '.join(risk['flags'])}")
    print(f"📈 Transactions: {analysis['transaction_counts']}")

def custom_trigger_example(address: str, quick_check: Dict) -> Dict:
    """Example custom trigger function"""
    # Trigger if >30 token transactions in 1 hour
    if quick_check.get("token_1h", 0) > 30:
        return {
            "triggered": True,
            "reason": f"High token activity: {quick_check['token_1h']} token txs/hour"
        }
    return {"triggered": False}

if __name__ == "__main__":
    # Initialize reactive detector
    API_KEY = "YOUR_ETHERSCAN_API_KEY"
    detector = ReactiveEtherscanAnomalyDetector(API_KEY)
    
    # Add alert callback
    detector.add_alert_callback(reactive_alert_callback)
    
    # Add custom triggers
    detector.add_anomaly_trigger(custom_trigger_example)
    
    # Start reactive monitoring (lightweight)
    addresses_to_monitor = [
        "0xE592427A0AEce92De3Edee1F18E0157C05861564",  # Uniswap V3
        "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D",  # Uniswap V2
    ]
    
    detector.start_reactive_monitoring(addresses_to_monitor)
    
    print(f"🎯 REACTIVE MONITORING ACTIVE")
    print(f"📊 Monitoring {len(addresses_to_monitor)} addresses")
    print(f"⚡ Will fetch detailed data ONLY when anomalies detected")
    print(f"🔄 Lightweight checks every 5 minutes")
    print(f"📅 Analysis window: Last 48 hours")
    
    try:
        while True:
            time.sleep(300)  # Check status every 5 minutes
            print(f"📡 Cache entries: {len(detector.address_cache)}")
            print(f"🔢 Daily API calls: {detector.daily_request_count}")
    except KeyboardInterrupt:
        detector.stop_reactive_monitoring()
        print("\n✅ Reactive monitoring stopped")